In [36]:
%matplotlib inline
import numpy as np
import numpy.ma as ma
import pandas as pd
from pandas import DataFrame, Series
import matplotlib.pyplot as plt
import seaborn as sns
import timeit
import line_profiler

import datetime

from io import StringIO
from pandas.api.types import CategoricalDtype

import pandas._testing as tm

## returns df from string data
def gdf(data):
    return pd.read_csv(StringIO(data))

In [36]:
a_df.set_index(0).stack().to_frame().reset_index(level=1)

,level_1,0
0,,
19,1,a
20,1,d
31,1,d
31,1,c
51,1,d


In [110]:
a_df.set_index(0).stack().to_frame().reset_index(level=1).rename(
    columns={(0):'col'}).pivot_table(
    columns='col', values='level_1', index=a_df)

,level_1,col
0,,
19,1,a
20,1,d
31,1,d
31,1,c
51,1,d


In [111]:
data = """Name,StartPoint,EndPoint,isDelivered,Customer
A,1,4,0,C1
A,1,4,0,C1
A,2,5,1,C1
A,3,5,0,C1
A,3,6,0,C1
A,3,6,1,C1
B,1,4,0,C2
B,1,4,0,C2
B,2,5,1,C2
B,3,5,1,C2
B,3,6,1,C2
B,3,8,0,C2
B,3,8,1,C2"""

df=pd.read_csv(StringIO(data))
df

,Name,StartPoint,EndPoint,isDelivered,Customer
0,A,1,4,0,C1
1,A,1,4,0,C1
2,A,2,5,1,C1
3,A,3,5,0,C1
4,A,3,6,0,C1
5,A,3,6,1,C1
6,B,1,4,0,C2
7,B,1,4,0,C2
8,B,2,5,1,C2
9,B,3,5,1,C2


In [126]:
df.groupby('Name')['StartPoint'].min()

Name
A    1
B    1
Name: StartPoint, dtype: int64

In [131]:
data = """[
   {
      "id":"abcd",
      "events":[
         {
            "description":"Liquidated",
            "time":"2020-12-18T12:00:00Z",
            "origin":{
               "name":"abc, GA",
               "code":" "
            },
            "destination":{
               "name":" ",
               "code":"SAV"
            },
            "remarks":"L 99",
            "invoice":"pqr",
            "code":"LIQ"
         },
         {
            "description":"Customs Monthly Stmt Paid",
            "time":"2020-02-26T12:00:00Z",
            "origin":{
               
            },
            "destination":{
               "name":"SAVANNAH, GA",
               "code":"SAV"
            },
            "remarks":"PM Stmt No 1720P0207Q",
            "invoice":"pqr",
            "code":"MSP"
         }
         ]
  },
  {
      "id":"xyz",
      "events":[
         {
            "description":"Liquidated",
            "time":"2020-12-18T12:00:00Z",
            "origin":{
               "name":"abc, GA",
               "code":" "
            },
            "destination":{
               "name":"abc",
               "code":""
            },
            "remarks":"L 99",
            "invoice":" ",
            "code":"LIQ"
         },
         {
            "description":"Customs Monthly Stmt Paid",
            "time":"2020-02-26T12:00:00Z",
            "origin":{
               
            },
            "destination":{
               "name":"SAVANNAH, GA",
               "code":"SAV"
            },
            "remarks":"PM Stmt No 1720P0207Q",
            "invoice":" ",
            "code":"MSP"
         }
         ]
}
]"""
df = pd.read_json(StringIO(data))
df.events.get(1)

[{'description': 'Liquidated',
  'time': '2020-12-18T12:00:00Z',
  'origin': {'name': 'abc, GA', 'code': ' '},
  'destination': {'name': 'abc', 'code': ''},
  'remarks': 'L 99',
  'invoice': ' ',
  'code': 'LIQ'},
 {'description': 'Customs Monthly Stmt Paid',
  'time': '2020-02-26T12:00:00Z',
  'origin': {},
  'destination': {'name': 'SAVANNAH, GA', 'code': 'SAV'},
  'remarks': 'PM Stmt No 1720P0207Q',
  'invoice': ' ',
  'code': 'MSP'}]

In [442]:
data="""Date,Tutor,Student
2021-04-12,310,112
2021-04-13,394,210
2021-04-14,357,3
2021-04-15,359,0
2021-04-16,392,0
2021-04-17,307,0
2021-04-18,335,0
2021-04-19,0,121"""
df=pd.read_csv(StringIO(data))
df = df.set_index('Date')
df = df.rename_axis('Name', axis=1)
df

Name,Tutor,Student
Date,,
2021-04-12,310,112
2021-04-13,394,210
2021-04-14,357,3
2021-04-15,359,0
2021-04-16,392,0
2021-04-17,307,0
2021-04-18,335,0
2021-04-19,0,121


In [446]:
df.iloc[:].apply(pd.to_timedelta, unit='s') 

Name,Tutor,Student
Date,,
2021-04-12,0 days 00:05:10,0 days 00:01:52
2021-04-13,0 days 00:06:34,0 days 00:03:30
2021-04-14,0 days 00:05:57,0 days 00:00:03
2021-04-15,0 days 00:05:59,0 days 00:00:00
2021-04-16,0 days 00:06:32,0 days 00:00:00
2021-04-17,0 days 00:05:07,0 days 00:00:00
2021-04-18,0 days 00:05:35,0 days 00:00:00
2021-04-19,0 days 00:00:00,0 days 00:02:01


In [158]:
pd.to_timedelta(df.Tutor.astype(str).add_suffix('s'))

Date
2021-04-12s   0 days 00:00:00.000000310
2021-04-13s   0 days 00:00:00.000000394
2021-04-14s   0 days 00:00:00.000000357
2021-04-15s   0 days 00:00:00.000000359
2021-04-16s   0 days 00:00:00.000000392
2021-04-17s   0 days 00:00:00.000000307
2021-04-18s   0 days 00:00:00.000000335
2021-04-19s             0 days 00:00:00
Name: Tutor, dtype: timedelta64[ns]

In [159]:
df.iloc[:].apply(pd.to_timedelta)

Name,Tutor,Student
Date,,
2021-04-12,0 days 00:00:00.000000310,0 days 00:00:00.000000112
2021-04-13,0 days 00:00:00.000000394,0 days 00:00:00.000000210
2021-04-14,0 days 00:00:00.000000357,0 days 00:00:00.000000003
2021-04-15,0 days 00:00:00.000000359,0 days 00:00:00
2021-04-16,0 days 00:00:00.000000392,0 days 00:00:00
2021-04-17,0 days 00:00:00.000000307,0 days 00:00:00
2021-04-18,0 days 00:00:00.000000335,0 days 00:00:00
2021-04-19,0 days 00:00:00,0 days 00:00:00.000000121


In [163]:
df.iloc[:].apply(pd.to_timedelta)

Name,Tutor,Student
Date,,
2021-04-12,0 days 00:00:00.000000310,0 days 00:00:00.000000112
2021-04-13,0 days 00:00:00.000000394,0 days 00:00:00.000000210
2021-04-14,0 days 00:00:00.000000357,0 days 00:00:00.000000003
2021-04-15,0 days 00:00:00.000000359,0 days 00:00:00
2021-04-16,0 days 00:00:00.000000392,0 days 00:00:00
2021-04-17,0 days 00:00:00.000000307,0 days 00:00:00
2021-04-18,0 days 00:00:00.000000335,0 days 00:00:00
2021-04-19,0 days 00:00:00,0 days 00:00:00.000000121


In [165]:
df.iloc[:].apply(pd.to_timedelta) 

Name,Tutor,Student
Date,,
2021-04-12,0 days 00:00:00.000000310,0 days 00:00:00.000000112
2021-04-13,0 days 00:00:00.000000394,0 days 00:00:00.000000210
2021-04-14,0 days 00:00:00.000000357,0 days 00:00:00.000000003
2021-04-15,0 days 00:00:00.000000359,0 days 00:00:00
2021-04-16,0 days 00:00:00.000000392,0 days 00:00:00
2021-04-17,0 days 00:00:00.000000307,0 days 00:00:00
2021-04-18,0 days 00:00:00.000000335,0 days 00:00:00
2021-04-19,0 days 00:00:00,0 days 00:00:00.000000121


In [193]:
df = DataFrame({'a':['x','x','y','y'], 'b':['1000','2000','3000','4000'],
               'c':[100,30,300,20]})
df

,a,b,c
0,x,1000,100
1,x,2000,30
2,y,3000,300
3,y,4000,20


In [194]:
df.set_index(['a','b']).reset_index().groupby('a', as_index=False).last()

,a,b,c
0,x,2000,30
1,y,4000,20


In [196]:
df1 = pd.DataFrame({'ID': ['AAA','AAA', 'AAA','BBB','CCC'], 'Buydate': ['2017-04-13', '2019-12-31', '2019-03-05', '2018-11-04', '2019-12-31' ], 'Quantity': [100.00, 2000.00, 385.95, 214514.00, 63205.00]}) 
df1 = df1.sort_values('Buydate')
#df1 = df1.set_index(['ID', 'Buydate']).reset_index()
df1['Buydate'] = pd.to_datetime(df1.Buydate)
df1

KeyError: "None of ['ROUGE_L'] are in the columns"

In [242]:
import random
# input datast
length = 10
ts = pd.date_range(start='2021/01/01 00:00', periods=length, freq='1h')
random.seed(1)
val = random.sample(range(1, 10+length), length)
df = pd.DataFrame({'val' : val}, index=ts)

# groupby custom datetimeindex
key_ts = [ts[i] for i in [1,3,7]]
df.loc[key_ts, 'id'] = range(len(key_ts))
df['id'] = df['id'].ffill()
df

,val,id
2021-01-01 00:00:00,5,NaN
2021-01-01 01:00:00,3,0.0
2021-01-01 02:00:00,9,0.0
2021-01-01 03:00:00,4,1.0
2021-01-01 04:00:00,8,1.0
2021-01-01 05:00:00,13,1.0
2021-01-01 06:00:00,15,1.0
2021-01-01 07:00:00,14,2.0
2021-01-01 08:00:00,11,2.0
2021-01-01 09:00:00,7,2.0


In [243]:
df['id'] = df.id.astype('category')

In [245]:
%timeit df.groupby('id')['val'].cumsum()

580 µs ± 16.7 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [236]:
%timeit df.groupby('id')['val'].cumsum()

671 µs ± 14.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [209]:
df.groupby('id')['val'].cumsum()

2021-01-01 00:00:00    -1
2021-01-01 01:00:00     3
2021-01-01 02:00:00    12
2021-01-01 03:00:00     4
2021-01-01 04:00:00    12
2021-01-01 05:00:00    25
2021-01-01 06:00:00    40
2021-01-01 07:00:00    14
2021-01-01 08:00:00    25
2021-01-01 09:00:00    32
Freq: H, Name: val, dtype: int64

In [226]:
df['cumsum'] = df.groupby('id', sort=False)['val'].cumsum()
df

,val,id,cumsum
2021-01-01 00:00:00,5,NaN,0
2021-01-01 01:00:00,3,0.0,3
2021-01-01 02:00:00,9,0.0,12
2021-01-01 03:00:00,4,1.0,4
2021-01-01 04:00:00,8,1.0,12
2021-01-01 05:00:00,13,1.0,25
2021-01-01 06:00:00,15,1.0,40
2021-01-01 07:00:00,14,2.0,14
2021-01-01 08:00:00,11,2.0,25
2021-01-01 09:00:00,7,2.0,32


In [313]:
df = pd.DataFrame({
    'ID':np.repeat(101,6),
    'Currency':np.repeat('INR',6),
    'Val1':np.arange(1,7),
    'Val2':np.arange(7,13),
    'Month':pd.period_range(start='2020-10', end='2021-03', freq='M')[::-1]
})
df

,ID,Currency,Val1,Val2,Month
0,101,INR,1,7,2021-03
1,101,INR,2,8,2021-02
2,101,INR,3,9,2021-01
3,101,INR,4,10,2020-12
4,101,INR,5,11,2020-11
5,101,INR,6,12,2020-10


In [276]:
pd.pivot(df.set_index(['ID','Currency'])

Month                                             
val2                    7        8        9        10       11       12
ID  Currency val1                                                      
101 INR      1     2021-03      NaT      NaT      NaT      NaT      NaT
             2         NaT  2021-02      NaT      NaT      NaT      NaT
             3         NaT      NaT  2021-01      NaT      NaT      NaT
             4         NaT      NaT      NaT  2020-12      NaT      NaT
             5         NaT      NaT      NaT      NaT  2020-11      NaT
             6         NaT      NaT      NaT      NaT      NaT  2020-10

In [254]:
list(reversed(pd.period_range(start='2020-10', end='2021-03', freq='M')))

[Period('2021-03', 'M'),
 Period('2021-02', 'M'),
 Period('2021-01', 'M'),
 Period('2020-12', 'M'),
 Period('2020-11', 'M'),
 Period('2020-10', 'M')]

In [311]:
df.set_index(['ID','Currency','Month']).stack().unstack(-2).reset_index()

Month,ID,Currency,level_2,2020-10,2020-11,2020-12,2021-01,2021-02,2021-03
0,101,INR,val1,6,5,4,3,2,1
1,101,INR,val2,12,11,10,9,8,7


In [330]:
pd.pivot(
        df,
        index=["ID", "Currency"],
        columns="Month",
        values=["Val1", "Val2"],
    ).stack(0)

Month              2020-10  2020-11  2020-12  2021-01  2021-02  2021-03
ID  Currency                                                           
101 INR      Val1        6        5        4        3        2        1
             Val2       12       11       10        9        8        7

In [334]:
data = """id,grouping_val
1,a
2,a
3,b
4,b"""
df = pd.read_csv(StringIO(data))
df['data'] = [[[0, 1], [1, 0]], [[1, 0], [0, 1]], [[2, 0], [3, 0]],
             [[0, 4], [4, 5]]]
df

,id,grouping_val,data
0,1,a,"[[0, 1], [1, 0]]"
1,2,a,"[[1, 0], [0, 1]]"
2,3,b,"[[2, 0], [3, 0]]"
3,4,b,"[[0, 4], [4, 5]]"


In [344]:
np.flat(df.data)

AttributeError: module 'numpy' has no attribute 'flat'

In [376]:
df = pd.DataFrame({'date': ['2017-04-13', '2019-5-31', '2019-03-05', '2018-3-04', '2019-7-31' ], 'Quantity': [100.00, 2000.00, 385.95, 214514.00, 63205.00]}) 
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5 entries, 0 to 4
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   date      5 non-null      object 
 1   Quantity  5 non-null      float64
dtypes: float64(1), object(1)
memory usage: 208.0+ bytes


In [370]:
df['date']=pd.to_datetime(df.date)
#df = df.set_index('date')


In [371]:
#df.loc['2013-01-01':'2019-12-31']

In [372]:
mask = (df['date'] > '2017-04-13') & (df['date'] <= '2019-04-13')
df.loc[mask]

,date,Quantity
2,2019-03-05,385.95
3,2018-03-04,214514.00


In [378]:
mask = (df['Date'] > '01-01-2013') & (df['Date'] <= '31-12-2018')
df.loc[mask]

KeyError: 'Date'

In [377]:
df[mask]

,date,Quantity
0,2017-04-13,100.0
3,2018-3-04,214514.0


In [393]:
dates=['2017-12-01','2017-12-05','2017-12-17','2017-12-28','2018-01-05','2018-01-09']
val=[5,2,3,6,1,7]
df = pd.DataFrame(list(zip(dates, val)),
               columns =['date', 'val'])
df['date'] = pd.to_datetime(df.date)
df

,date,val
0,2017-12-01,5
1,2017-12-05,2
2,2017-12-17,3
3,2017-12-28,6
4,2018-01-05,1
5,2018-01-09,7


In [422]:
df['week'] = 1+(df['date']-df.loc[0,'date']).dt.days // 7
df

,date,val,week
0,2017-12-01,5,1
1,2017-12-05,2,1
2,2017-12-17,3,3
3,2017-12-28,6,4
4,2018-01-05,1,6
5,2018-01-09,7,6


In [440]:
data = """created,id
1601820360,10
1601820365,11"""
df =pd.read_csv(StringIO(data))
df

,created,id
0,1601820360,10
1,1601820365,11


In [441]:
df['created'] = pd.to_datetime(df.created, unit='s')
df

,created,id
0,2020-10-04 14:06:00,10
1,2020-10-04 14:06:05,11


In [469]:
df = DataFrame({
    'a':['x','x','y','x','y','z'],
    'b':np.arange(1,7),
    'c':np.arange(1,7)
})



In [470]:
df.groupby('a', as_index=False)[['b','c']].count()


,a,b,c
0,x,3,3
1,y,2,2
2,z,1,1


In [477]:
data = """created_at, value1,value2,value3
2021-04-25 11:38:33,1,1,5
2021-04-25 11:38:47,4,3,6
2021-04-25 11:39:36,1,1,8 
2021-04-25 11:39:47,6,5,5 
2021-04-25 11:40:50,8,7,3"""
df = pd.read_csv(StringIO(data))
df['created_at'] = pd.to_datetime(df.created_at)
df

,created_at,value1,value2,value3
0,2021-04-25 11:38:33,1,1,5
1,2021-04-25 11:38:47,4,3,6
2,2021-04-25 11:39:36,1,1,8
3,2021-04-25 11:39:47,6,5,5
4,2021-04-25 11:40:50,8,7,3


In [478]:
pd.DataFrame(df.groupby([pd.Grouper(key='created_at', freq='2Min')]).mean())

,value1,value2,value3
created_at,,,
2021-04-25 11:38:00,3.0,2.5,6.0
2021-04-25 11:40:00,8.0,7.0,3.0


In [481]:
data="""date,id
2020-10-04 14:06:00,10.0
2020-10-04 14:06:05,11.0 
2020-10-05 12:28:58,12.0
2020-10-05 12:29:16,13.0
2020-10-05 12:29:31,14.0"""
df = pd.read_csv(StringIO(data))
df['date'] = pd.to_datetime(df.date)
df

,date,id
0,2020-10-04 14:06:00,10.0
1,2020-10-04 14:06:05,11.0
2,2020-10-05 12:28:58,12.0
3,2020-10-05 12:29:16,13.0
4,2020-10-05 12:29:31,14.0


In [487]:
pd.DataFrame(df.groupby([pd.Grouper(key='date', freq='15Min')]).sum()).reset_index().head()

,date,id
0,2020-10-04 14:00:00,21.0
1,2020-10-04 14:15:00,0.0
2,2020-10-04 14:30:00,0.0
3,2020-10-04 14:45:00,0.0
4,2020-10-04 15:00:00,0.0


In [576]:
df = pd.DataFrame({"col_1": ["apple", "banana", "apple", "banana", "banana"],
                   "col_2": [1, 4, 8, 8, 6],
                   "col_3": [56, 4, 22, 1, 5]})
df

,col_1,col_2,col_3
0,apple,1,56
1,banana,4,4
2,apple,8,22
3,banana,8,1
4,banana,6,5


In [504]:
dfo = DataFrame({"col_1": ["apple", "banana"],
 "col_2": [[1, 8], [4, 8, 6]],
                 ] "col_3": [[56, 22], [4, 1, 5]],
 "count": [2,3]})
dfo

,col_1,col_2,col_3,count
0,apple,"[1, 8]","[56, 22]",2
1,banana,"[4, 8, 6]","[4, 1, 5]",3


In [564]:
df['count'] = df.groupby('col_1')['col_2'].transform('count')
df

,col_1,col_2,col_3,count
0,apple,1,56,2
1,banana,4,4,3
2,apple,8,22,2
3,banana,8,1,3
4,banana,6,5,3


In [586]:
%timeit df.groupby('col_1').agg({ 'col_2': lambda x: list(x),'col_3': lambda x: list(x),}).reset_index().join(df.groupby('col_1')['col_2'].transform('count').rename('count'))

6.11 ms ± 157 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [587]:
%timeit df.groupby('col_1')[['col_2', 'col_3']].agg(list).join(df.groupby('col_1').size().rename('count'))

9.09 ms ± 217 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [4]:
df = pd.DataFrame({
    'Year':np.tile([1953, 1954], 4),
    'Month':np.tile([1,2], 4),
    'MaxTemperature':[26.7, 29.2, 27.4, 28, 28, 25.4, 26, 29.1],
    'MinTemperature':[10.9, 9.9, 11, 11.2, 13, 9.4, 12, 10.5]
})
df

,Year,Month,MaxTemperature,MinTemperature
0,1953,1,26.7,10.9
1,1954,2,29.2,9.9
2,1953,1,27.4,11.0
3,1954,2,28.0,11.2
4,1953,1,28.0,13.0
5,1954,2,25.4,9.4
6,1953,1,26.0,12.0
7,1954,2,29.1,10.5


In [7]:
df.groupby('Year',as_index=False).mean().drop(columns='Month')

,Year,MaxTemperature,MinTemperature
0,1953,27.025,11.725
1,1954,27.925,10.250


In [8]:
data = ["2/300/3" , "4/300/3"]
df =pd.DataFrame(data, columns=["data"])
df["sum"] = df["data"].str.replace('/','+').astype(int).sum()

ValueError: invalid literal for int() with base 10: '2+300+3'

In [406]:
d = """Real First Packet,doctets,Duration
2013-02-04 15:45:00,456,64
2013-02-04 15:50:00,282111,56576
2013-02-04 16:00:00,271426,173632
2013-02-04 16:05:00,0,0
2013-02-15 16:20:00,0,0
2013-02-15 16:25:00,0,0
2013-02-15 16:30:00,47255,6656"""

df = pd.read_csv(StringIO(d))
df['Real First Packet'] = pd.to_datetime(df['Real First Packet'])
#df = df.set_index('Real First Packet')
#df = df.index.astype(datetime64[ns])
#df = df.resample('5min').asfreq()
df['Real First Packet'].dt.time
pd.to_datetime('00:00:00')

Timestamp('2021-04-26 00:00:00')

In [283]:
df = df.resample('5min').asfreq().reindex(pd.date_range('2013-02-04 8:00:00', 
                         '2013-02-15 17:00:00', 
                        freq='5 min')).fillna(0).astype(int)
df.tail(7)

,doctets,Duration
2013-02-15 16:30:00,47255,6656
2013-02-15 16:35:00,0,0
2013-02-15 16:40:00,0,0
2013-02-15 16:45:00,0,0
2013-02-15 16:50:00,0,0
2013-02-15 16:55:00,0,0
2013-02-15 17:00:00,0,0


In [79]:
df1 = pd.DataFrame({
    'RealFirstPacket':pd.date_range(start='2013-02-04 8:00:00', end='2013-02-15 17:00:00')
})
df1['RealFirstPacket'] = pd.to_datetime(df1.RealFirstPacket)
df1 = df1.set_index('RealFirstPacket')
df1 = df1.resample('5min').asfreq()

In [67]:
dft = pd.merge_asof(
    df1, df, left_index=True, right_index=True, allow_exact_matches=True).fillna(0).astype(int)

In [68]:
dft.to_csv('te.csv')

In [76]:
dft = pd.concat([df, df1], axis=1, join='outer')
dft.to_csv('te.csv')

In [243]:
dft = df.merge(
    pd.date_range(start='2013-02-04 8:00:00',
                  end='2013-02-15 17:00:00',
                  freq='5min',name='Real First Packet').to_frame()\
    .set_index('Real First Packet'),
    left_index=True,
    right_index=True,
    how='right'
 ).fillna(0).astype(int)
dft.tail(7)

IndexError: single positional indexer is out-of-bounds

In [212]:
pd.date_range(start='2013-02-04 8:00:00',
                  end='2013-02-15 17:00:00',
                  freq='5min', name='RealFirstPacket')


DatetimeIndex(['2013-02-04 08:00:00', '2013-02-04 08:05:00',
               '2013-02-04 08:10:00', '2013-02-04 08:15:00',
               '2013-02-04 08:20:00', '2013-02-04 08:25:00',
               '2013-02-04 08:30:00', '2013-02-04 08:35:00',
               '2013-02-04 08:40:00', '2013-02-04 08:45:00',
               ...
               '2013-02-15 16:15:00', '2013-02-15 16:20:00',
               '2013-02-15 16:25:00', '2013-02-15 16:30:00',
               '2013-02-15 16:35:00', '2013-02-15 16:40:00',
               '2013-02-15 16:45:00', '2013-02-15 16:50:00',
               '2013-02-15 16:55:00', '2013-02-15 17:00:00'],
              dtype='datetime64[ns]', name='RealFirstPacket', length=3277, freq='5T')

In [207]:
dft = df.merge(
    pd.date_range(start='2013-02-04 8:00:00',
                  end='2013-02-15 17:00:00',
                  freq='5min').to_frame(
        name='RealFirstPacket').set_index('RealFirstPacket'),
    left_index=True,
    right_index=True,
    how='right'
 ).fillna(0).astype(int)
dft.tail(7)

,id,related_id,type
RealFirstPacket,,,
2013-02-15 16:30:00,0,0,0
2013-02-15 16:35:00,0,0,0
2013-02-15 16:40:00,0,0,0
2013-02-15 16:45:00,0,0,0
2013-02-15 16:50:00,0,0,0
2013-02-15 16:55:00,0,0,0
2013-02-15 17:00:00,0,0,0


In [100]:
dft.to_csv('te.csv')

In [114]:
df = pd.read_csv('data/tips.csv')
pd.pivot_table(df, values='total_bill',index='day', 
               columns='time', aggfunc='max').loc[['Sat', 'Sun']]

time,Dinner,Lunch
day,,
Sat,50.81,NaN
Sun,48.17,NaN


In [126]:
d = """Form Date,Form Date.2,Form Date.3
2021-01-05,,
2021-02-05,,
,,2021-03-02
,2021-03-22,
2021-01-06,2021-01-06,
,,2021-02-12
2021-01-09,,
,21-02-19,
2021-01-09,2021-01-09,"""
df = pd.read_csv(StringIO(d))
df['Form Date'] = pd.to_datetime(df['Form Date'])
df['Form Date.2'] = pd.to_datetime(df['Form Date.2'])
df['Form Date.3'] = pd.to_datetime(df['Form Date.3'])
df

,Form Date,Form Date.2,Form Date.3
0,2021-01-05,NaT,NaT
1,2021-02-05,NaT,NaT
2,NaT,NaT,2021-03-02
3,NaT,2021-03-22,NaT
4,2021-01-06,2021-01-06,NaT
5,NaT,NaT,2021-02-12
6,2021-01-09,NaT,NaT
7,NaT,2019-02-21,NaT
8,2021-01-09,2021-01-09,NaT


In [137]:
#df['Merged Form Dates'] = df['Form Date']+df['Form Date.2']
column=['Form Date','Form Date.2','Form Date.3']
df[column].astype(str).apply(set,1).apply(''.join).str.strip('NaT')

0    2021-01-05
1    2021-02-05
2    2021-03-02
3    2021-03-22
4    2021-01-06
5    2021-02-12
6    2021-01-09
7    2019-02-21
8    2021-01-09
dtype: object

In [139]:
datetime.datetime.combine(column)

TypeError: combine() argument 1 must be datetime.date, not list

In [143]:
d = """Place,Name,2019-03-01,2019-03-02,2021-03-02
Paris,Sam,2,0,0
Paris,Steve,12,4,0
Dublin,Sam,8,1,0"""
df = pd.read_csv(StringIO(d))
df

,Place,Name,2019-03-01,2019-03-02,2021-03-02
0,Paris,Sam,2,0,0
1,Paris,Steve,12,4,0
2,Dublin,Sam,8,1,0


In [150]:
df.set_index(['Place', 'Name']).stack().unstack(1)

Name               Sam  Steve
Place                        
Dublin 2019-03-01  8.0    NaN
       2019-03-02  1.0    NaN
       2021-03-02  0.0    NaN
Paris  2019-03-01  2.0   12.0
       2019-03-02  0.0    4.0
       2021-03-02  0.0    0.0

In [151]:
d = """Products,Price
A,100
B,500
C,1000
D,3000
E,6000"""
df =pd.read_csv(StringIO(d))
df

,Products,Price
0,A,100
1,B,500
2,C,1000
3,D,3000
4,E,6000


In [164]:
np.where((df.Price>=1000) & (df.Price<=5000),
         (.9*df.Price),df.Price)

array([ 100.,  500.,  900., 2700., 6000.])

In [161]:
df.append(pd.DataFrame({'Products':df.Products, 'Price':.1*df.Price}))

,Products,Price
0,A,100.0
1,B,500.0
2,C,1000.0
3,D,3000.0
4,E,6000.0
0,A,10.0
1,B,50.0
2,C,100.0
3,D,300.0
4,E,600.0


In [195]:
l = ['1197608_b', '1100_b', '1200', 'b_1300']
df = pd.DataFrame(l, columns=['input'])
df

,input
0,1197608_b
1,1100_b
2,1200
3,b_1300


In [197]:
df['nums'] = df.input.str.extract(r'(\d+)')
df['chars'] = df.input.str.extract(r'([a-zA-Z])')

In [199]:
df.nums.to_list()

['1197608', '1100', '1200', '1300']

In [200]:
class class1: 
    def __init__(self, m, x, y, z): 
        self.m = m
        self.x = x #int
        self.y = y #int
        self.z = z #int

#creating list       
list = [] 
compare = []  

# appending instances to list 
list.append( class1( m, x, y, z) )
list.append( class1( m, x, y, z) )
list.append( class1( m, x, y, z) )

list

NameError: name 'm' is not defined

In [245]:
data="""id,related_id,type
1,2,desktop
2,1,mobile
3,NaN,desktop
4,6,desktop
5,NaN,mobile
6,4,mobile"""
df = pd.read_csv(StringIO(data))
df

,id,related_id,type
0,1,2.0,desktop
1,2,1.0,mobile
2,3,NaN,desktop
3,4,6.0,desktop
4,5,NaN,mobile
5,6,4.0,mobile


In [202]:
df.query('related_id.isna() | type=="desktop"')

,id,related_id,type
0,1,2.0,desktop
2,3,NaN,desktop
3,4,6.0,desktop
4,5,NaN,mobile


In [205]:
dataDict = {z[0]: list(z[1:]) for z in zip(date, prcp, tMax, tMin, rain)}
print(dataDict)



NameError: name 'date' is not defined

In [254]:
list = np.arange(21)
print(list) # [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]
list[10::10] # array([10, 20])

[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20]


array([10, 20])

In [255]:
d = """id,date,amt,rate
15856,2015-12-31,85.09,0.0175
15857,2016-03-31,135.60,0.0175
15858,2016-06-30,135.91,0.0175
15859,2016-09-30,167.27,0.0175
15860,2016-12-31,173.32,0.0175
15875,2020-09-30,305.03,0.0175
15876,2020-12-31,354.09,0.0175
15877,2021-03-31,391.19,0.0175"""
df = pd.read_csv(StringIO(d))

In [256]:
df1 = pd.DataFrame({
    id=np.arange(15856, 15878),
    date=pd.
})

,id,date,amt,rate
0,15856,2015-12-31,85.09,0.0175
1,15857,2016-03-31,135.60,0.0175
2,15858,2016-06-30,135.91,0.0175
3,15859,2016-09-30,167.27,0.0175
4,15860,2016-12-31,173.32,0.0175
5,15875,2020-09-30,305.03,0.0175
6,15876,2020-12-31,354.09,0.0175
7,15877,2021-03-31,391.19,0.0175


In [257]:
pd.Period(freq='Q')

DateParseError: Unknown datetime string format, unable to parse: ID,DATE,AMT,RATE
15856,2015-12-31,85.09,0.0175
15857,2016-03-31,135.60,0.0175
15858,2016-06-30,135.91,0.0175
15859,2016-09-30,167.27,0.0175
15860,2016-12-31,173.32,0.0175
15875,2020-09-30,305.03,0.0175
15876,2020-12-31,354.09,0.0175
15877,2021-03-31,391.19,0.0175

In [258]:
df = pd.DataFrame({
    'A': np.tile(['a','b','c'], 3),
    'B':['b','c','d','a','c','d','a','b','d'],
    'dis':[1,2,3,1,5,6,2,5,4]
})
df

,A,B,dis
0,a,b,1
1,b,c,2
2,c,d,3
3,a,a,1
4,b,c,5
5,c,d,6
6,a,a,2
7,b,b,5
8,c,d,4


In [266]:
df.groupby('A', as_index=False)['dis'].nsmallest(2)

AttributeError: 'DataFrameGroupBy' object has no attribute 'nsmallest'

In [271]:
df.groupby(['A'], as_index=False).min(2)

,A,dis
0,a,1
1,b,2
2,c,3


In [284]:
d = {'col': [True, False, True, True, False, True, False]}

df = pd.DataFrame(data=d)
df

,col
0,True
1,False
2,True
3,True
4,False
5,True
6,False


In [285]:
df1 = pd.DataFrame({'type': ['cst1', 'cst1', 'cst2','cst1','cst2','cst3','cst2','cst1','cst2','cst4','cst5','cst3'],
                   'year': [2017, 2017, 2017,2018,2018,2018,2018,2019,2019,2019,2019,2020]})
df1.groupby('type', as_index=False).

,type,year
0,cst1,2017
1,cst1,2017
2,cst2,2017
3,cst1,2018
4,cst2,2018
5,cst3,2018
6,cst2,2018
7,cst1,2019
8,cst2,2019
9,cst4,2019


In [290]:
for i,j in df.iterrows():
    print(i)

0
1
2
3
4
5
6


In [295]:
df = pd.DataFrame({'x': [0, 0, 0, 0, 1, 1, 1, 1], 
                    'y': ['a','b','c','d','e','f','g','h']})
df

,x,y
0,0,a
1,0,b
2,0,c
3,0,d
4,1,e
5,1,f
6,1,g
7,1,h


In [305]:
%timeit df.groupby('x', as_index=False)['y'].apply(lambda x: (x.shift(fill_value='') + ' ').cumsum().str.strip()).reset_index(drop=True)

5.13 ms ± 731 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [308]:
%timeit df.groupby('x')['y'].transform(lambda x: (x.shift(fill_value='') + ',').cumsum()).str.strip(',')

2.71 ms ± 315 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [323]:
df1 = pd.DataFrame({
    'A':'Dell  Lenovo Acer Apple Lenovo Dell'.split()
})
df1

df2 = df1.value_counts().reset_index().rename(columns={(0):'B'})
df2

,A,B
0,Dell,2
1,Lenovo,2
2,Acer,1
3,Apple,1


In [317]:
df.value_counts()

x  y
0  a    1
   b    1
   c    1
   d    1
1  e    1
   f    1
   g    1
   h    1
dtype: int64

In [342]:
d = """ID,Date,Value
1,8/9/2018,2857
1,8/15/2018,2194
1,8/23/2018,226
1,8/30/2018,685
2,8/6/2018,1390
2,8/17/2018,1162
2,8/27/2018,6320
2,8/30/2018,1150"""
df = pd.read_csv(StringIO(d))
df['Date'] = pd.to_datetime(df.Date)
df

,ID,Date,Value
0,1,2018-08-09,2857
1,1,2018-08-15,2194
2,1,2018-08-23,226
3,1,2018-08-30,685
4,2,2018-08-06,1390
5,2,2018-08-17,1162
6,2,2018-08-27,6320
7,2,2018-08-30,1150


In [354]:
df.groupby(['ID', pd.Grouper(key='Date', freq='1W')]).sum().reset_index(
).set_index('Date').reindex(pd.date_range('2018/7/1', '2020/07/31', freq='1W'))

ValueError: cannot reindex from a duplicate axis

In [357]:
d="""col_1
Agent AB 7:00 AM
Agent AB 7:00 AM
Cust XY 8:00 AM
Cust XY 9:00 AM
Agent AB 11:00 AM"""

df = pd.read_csv(StringIO(d))

df

,col_1
0,Agent AB 7:00 AM
1,Agent AB 7:00 AM
2,Cust XY 8:00 AM
3,Cust XY 9:00 AM
4,Agent AB 11:00 AM


In [390]:
df.col_1.str.split(" ", 2, expand=True)

,0,1,2
0,Agent,AB,7:00 AM
1,Agent,AB,7:00 AM
2,Cust,XY,8:00 AM
3,Cust,XY,9:00 AM
4,Agent,AB,11:00 AM


In [383]:
df.col_1.str.extract('(.*,\s[A-Z]{2})(.*)')

,0,1
0,NaN,NaN
1,NaN,NaN
2,NaN,NaN
3,NaN,NaN
4,NaN,NaN


In [394]:
d = """Alert number,Age,Job,Loan
0,58,4,0
2,44,9,0
4,35,4,0
6,41,0,0
8,29,0,0"""
df1 = pd.read_csv(StringIO(d))
df1

,Alert number,Age,Job,Loan
0,0,58,4,0
1,2,44,9,0
2,4,35,4,0
3,6,41,0,0
4,8,29,0,0


In [395]:
d = """Alert number,Age,Job,Loan
1,58,4,0
3,44,9,0
5,35,4,0
7,41,0,0
9,29,0,0"""
df2 = pd.read_csv(StringIO(d))
df2

,Alert number,Age,Job,Loan
0,1,58,4,0
1,3,44,9,0
2,5,35,4,0
3,7,41,0,0
4,9,29,0,0


In [399]:
df1.append(df2).sort_values('Alert number').reset_index(drop=True)

,Alert number,Age,Job,Loan
0,0,58,4,0
1,1,58,4,0
2,2,44,9,0
3,3,44,9,0
4,4,35,4,0
5,5,35,4,0
6,6,41,0,0
7,7,41,0,0
8,8,29,0,0
9,9,29,0,0


In [400]:
df = pd.DataFrame(data, columns=["1","2","3","4","5","6","7","8","9"])

ValueError: DataFrame constructor not properly called!

In [420]:
raw_data = {'age1': [23,45,210],'age2': [10,20,150],'name': ['a','b','c']}
df = pd.DataFrame(raw_data, columns = ['age1','age2','name'])

raw_data = {'age1': [80,90,110],'age2': [70,120,90],'name': ['a','b','c']}
df2 = pd.DataFrame(raw_data, columns = ['age1','age2','name'])

In [421]:
df.where(df.age1.lt(100),'dead')

,age1,age2,name
0,23,10,a
1,45,20,b
2,dead,dead,dead


In [418]:
df2

,age1,age2,name
0,80,70,a
1,90,120,b
2,110,90,c


TypeError: '<' not supported between instances of 'str' and 'int'

In [415]:
df.age1

0      23
1      45
2    dead
Name: age1, dtype: object

In [448]:
d = """Name,Age
michael,18
lincoln,20
theodore,84
alexandre,95"""
df1 = pd.read_csv(StringIO(d))
df1

,Name,Age
0,michael,18
1,lincoln,20
2,theodore,84
3,alexandre,95


In [449]:
d = """Name,Age
sayed,17
hurley,29
sawyer,44
John,15"""
df2 = pd.read_csv(StringIO(d))
df2

,Name,Age
0,sayed,17
1,hurley,29
2,sawyer,44
3,John,15


In [450]:
d = """Name,Age
walter,50
jesse,15
fring,20
saul,34"""
df3 = pd.read_csv(StringIO(d))
df3

,Name,Age
0,walter,50
1,jesse,15
2,fring,20
3,saul,34


In [454]:
df1 = df1.set_index('Age')
df2 = df2.set_index('Age')
df3 = df3.set_index('Age')

In [444]:
pd.DataFrame(np.where(df1.index > df2.index,[df1.index, df1.Name],
         [df2.index, df2.Name])).T

,0,1
0,18,michael
1,29,hurley
2,84,theodore
3,95,alexandre


In [455]:
conditions = [
    ((df1.index > df2.index) & (df1.index > df3.index)),
    ((df2.index > df1.index) & (df2.index > df3.index)),
    ((df3.index > df1.index) & (df3.index > df3.index)),
]
choices = [[df1.Name, df1.index], [df2.Name, df2.index], [df3.Name, df3.index]]
np.select(conditions, choices)

array([[0, 'hurley', 'theodore', 'alexandre'],
       [0, 29, 84, 95]], dtype=object)

In [474]:
pd.to_timedelta('1 min 50 sec').seconds

110

In [472]:
df = pd.DataFrame({'time': ["1 min 50 secs", "2 mins 10 secs", "40 secs"]})
df

,time
0,1 min 50 secs
1,2 mins 10 secs
2,40 secs


In [487]:
d
df

,time,timedelta
0,1 min 50 secs,110
1,2 mins 10 secs,130
2,40 secs,40


In [488]:
d = """title,date,sentiment,stock_ticker
good news from vw,2021-04-01,positive,vlkaf
some post about vw,2021-04-01,positive,vlkaf
another post on vw,2021-04-01,negative,vlkaf
post about gme,2020-05-01,positive,gme
post about gme again,2020-05-01,negative,gme
another gme post,2020-05-01,negative,gme
more vw posts,2021-04-02,negative,vlkaf
and again vw,2021-04-02,negative,vlkaf"""
df = pd.read_csv(StringIO(d))
df

,title,date,sentiment,stock_ticker
0,good news from vw,2021-04-01,positive,vlkaf
1,some post about vw,2021-04-01,positive,vlkaf
2,another post on vw,2021-04-01,negative,vlkaf
3,post about gme,2020-05-01,positive,gme
4,post about gme again,2020-05-01,negative,gme
5,another gme post,2020-05-01,negative,gme
6,more vw posts,2021-04-02,negative,vlkaf
7,and again vw,2021-04-02,negative,vlkaf


In [489]:
df.groupby('stock_ticker').sum()

,title,date,sentiment
stock_ticker,,,
gme,post about gmepost about gme againanother gme ...,2020-05-012020-05-012020-05-01,positivenegativenegative
vlkaf,good news from vwsome post about vwanother pos...,2021-04-012021-04-012021-04-012021-04-022021-0...,positivepositivenegativenegativenegative


In [43]:
data = """Receipt / Payment,Amount
Payment,1233.0
Receipt,4561.3"""

df = gdf(data)
df

,Receipt / Payment,Amount
0,Payment,1233.0
1,Receipt,4561.3


In [44]:
df['Amount 2'] =  df.Amount.where(df['Receipt / Payment'].ne('Payment'), -df.Amount)
df

,Receipt / Payment,Amount,Amount 2
0,Payment,1233.0,-1233.0
1,Receipt,4561.3,4561.3


In [37]:
d = """titleId,x0_nm0000212,x0_nm0000630,x0_nm0003506,x0_nm0005227,x0_nm0413168
tt0035423,1.0,0.0,0.0,0.0,0.0
tt0035423,0.0,0.0,0.0,0.0,1.0
tt0035423,0.0,1.0,0.0,0.0,0.0
tt0035423,0.0,0.0,0.0,1.0,0.0
tt0035423,0.0,0.0,1.0,0.0,0.0"""
df1 = gdf(d)
df1 = df1.set_index('titleId')
df1

,x0_nm0000212,x0_nm0000630,x0_nm0003506,x0_nm0005227,x0_nm0413168
titleId,,,,,
tt0035423,1.0,0.0,0.0,0.0,0.0
tt0035423,0.0,0.0,0.0,0.0,1.0
tt0035423,0.0,1.0,0.0,0.0,0.0
tt0035423,0.0,0.0,0.0,1.0,0.0
tt0035423,0.0,0.0,1.0,0.0,0.0


In [35]:
df1.reset_index().groupby('titleId').max()

,x0_nm0000212,x0_nm0000630,x0_nm0003506,x0_nm0005227,x0_nm0413168
titleId,,,,,
tt0035423,1.0,1.0,1.0,1.0,1.0


In [58]:
pd.set_option('mode.chained_assignment','warn')
df = DataFrame({
    'year':['2010', '2011-1', '1999', '1998-1']
})
df['year'] = df.year.copy().str.split('-').str[0]
df

,year
0,2010
1,2011
2,1999
3,1998


In [71]:
df = pd.DataFrame([[1, "ABABBBAAAB"], [2, "BBABBBAAAA"], [3, "ABBAABABAB"]], columns=['StratID', 'Rand'])
df

,StratID,Rand
0,1,ABABBBAAAB
1,2,BBABBBAAAA
2,3,ABBAABABAB


In [64]:
df["trimmed_col"] = np.where(df["StratID"].isin({1, 3}), df.Rand.str[:6], "")
df

,StratID,Rand,trimmed_col
0,1,ABABBBAAAB,ABABBB
1,2,BBABBBAAAA,
2,3,ABBAABABAB,ABBAAB


In [65]:
df["trimmed_col"] = np.where(df["StratID"].isin({2}), df.Rand.str[:4], df["trimmed_col"])
df

,StratID,Rand,trimmed_col
0,1,ABABBBAAAB,ABABBB
1,2,BBABBBAAAA,BBAB
2,3,ABBAABABAB,ABBAAB


In [72]:
conditions = [
    df["StratID"].isin({1, 3}),
    df["StratID"].isin({2}),
]
choices = [df.Rand.str[:6], df.Rand.str[:4]]
df['trimmed_col'] = np.select(conditions, choices)
df

,StratID,Rand,trimmed_col
0,1,ABABBBAAAB,ABABBB
1,2,BBABBBAAAA,BBAB
2,3,ABBAABABAB,ABBAAB


In [132]:
d = """date, val
2018-01-05 00:00:00,2900.41
2018-01-05 00:01:00,2919.24
2018-01-05 00:02:00,2938.07
2018-01-08 00:00:00,3118.00
2018-01-08 00:01:00,3118.00
2018-01-08 00:02:00,3125.54"""
df = gdf(d)
df['date'] = pd.to_datetime(df.date)
df

,date,val
0,2018-01-05 00:00:00,2900.41
1,2018-01-05 00:01:00,2919.24
2,2018-01-05 00:02:00,2938.07
3,2018-01-08 00:00:00,3118.00
4,2018-01-08 00:01:00,3118.00
5,2018-01-08 00:02:00,3125.54


In [133]:
df = df.set_index('date')
df = df.reindex(pd.date_range('2018-01-05 00:00:00', 
                         '2018-01-08 00:02:00', freq='1 min')
          ).reset_index().rename(columns={'index':'date'})
df = df.groupby(df.date.dt.time).ffill()

In [135]:
df.to_csv('te.csv')

In [141]:
df.iloc[[0,1,2,1440,1441,1442,2880,2881,2882,4320,4321,4322]]

,date,val
0,2018-01-05 00:00:00,2900.41
1,2018-01-05 00:01:00,2919.24
2,2018-01-05 00:02:00,2938.07
1440,2018-01-06 00:00:00,2900.41
1441,2018-01-06 00:01:00,2919.24
1442,2018-01-06 00:02:00,2938.07
2880,2018-01-07 00:00:00,2900.41
2881,2018-01-07 00:01:00,2919.24
2882,2018-01-07 00:02:00,2938.07
4320,2018-01-08 00:00:00,3118.00


In [142]:
df1 = pd.DataFrame({ 'a':[12,11,35,48] , 
                    'b': [765, 221, 381, 213],
                    'c': [np.nan, np.nan, np.nan, np.nan]})
df2 = pd.DataFrame({'a':[12, 15, 11, 13, 35, 18, 48], 
                    'd':['Watch', 'Pen', 'Pencil', 'Eraser', 'bottle', 'box', 'toolkit'],
                    })
df1

,a,b,c
0,12,765,NaN
1,11,221,NaN
2,35,381,NaN
3,48,213,NaN


In [143]:
df2

,a,d
0,12,Watch
1,15,Pen
2,11,Pencil
3,13,Eraser
4,35,bottle
5,18,box
6,48,toolkit


In [144]:
df1.merge(df2, on='a', how='left')

,a,b,c,d
0,12,765,NaN,Watch
1,11,221,NaN,Pencil
2,35,381,NaN,bottle
3,48,213,NaN,toolkit


In [145]:
df = DataFrame({
    'year':['2010', '2011-1', '1999', '1998-1']
})

In [146]:
df['year1'] = df.year.str.split('-').str[0]

In [147]:
df

,year,year1
0,2010,2010
1,2011-1,2011
2,1999,1999
3,1998-1,1998


In [151]:
df = pd.DataFrame({
    'WT_RESIDUE':['ALA', "REMARK", 'VAL', "LYS"]
})
df

,WT_RESIDUE
0,ALA
1,REMARK
2,VAL
3,LYS


In [152]:
codes = {'ALA':'A', 'ARG':'R', 'ASN':'N', 'ASP':'D', 'CYS':'C', 'GLU':'E', 'GLN':'Q', 'GLY':'G', 'HIS':'H', 'ILE':'I', 'LEU':'L', 'LYS':'K', 'MET':'M', 'PHE':'F', 'PRO':'P', 'SER':'S', 'THR':'T', 'TRP':'W', 'TYR':'Y', 'VAL':'V'}
df['MUTATION_CODE'] = df.WT_RESIDUE.map(codes)
df

,WT_RESIDUE,MUTATION_CODE
0,ALA,A
1,REMARK,NaN
2,VAL,V
3,LYS,K


In [214]:
d = """ID,VAR1,VAR2
1,ABC,XYZ
2,MNP,QRS
3,UVW,STU"""
df = gdf(d)
df

,ID,VAR1,VAR2
0,1,ABC,XYZ
1,2,MNP,QRS
2,3,UVW,STU


In [215]:
df = df.loc[df.index.repeat(3)]
df.insert(1, 'YEAR_MONTH', df.ID.astype(str) + '_' + (df.groupby('ID', as_index=False).ID.cumcount()+1).astype(str))
df

,ID,YEAR_MONTH,VAR1,VAR2
0,1,1_1,ABC,XYZ
0,1,1_2,ABC,XYZ
0,1,1_3,ABC,XYZ
1,2,2_1,MNP,QRS
1,2,2_2,MNP,QRS
1,2,2_3,MNP,QRS
2,3,3_1,UVW,STU
2,3,3_2,UVW,STU
2,3,3_3,UVW,STU


In [210]:
df.insert(1, 'YEAR_MONTH', df.ID.astype(str) + '_' + (df.groupby('ID', as_index=False).ID.cumcount()+1).astype(str))

In [206]:
df

,ID,YEAR_MONTH,VAR1,VAR2
0,1,1_1,ABC,XYZ
1,1,1_2,ABC,XYZ
2,1,1_3,ABC,XYZ
3,2,2_1,MNP,QRS
4,2,2_2,MNP,QRS
5,2,2_3,MNP,QRS
6,3,3_1,UVW,STU
7,3,3_2,UVW,STU
8,3,3_3,UVW,STU


In [307]:
import pandas as pd
import numpy 
import io  


datastring = """
ride_id,start_time,end_time,driver_id,region,is_completed
4,5,2021-01-15 21:02:58,NaN,2,Cape Town,1
26,27,2021-03-31 21:51:00,NaN,2,San Francisco,1
0,1,2021-04-07 10:31:41,NaN,4,San Francisco,1
23,24,2021-02-20 06:37:07,NaN,4,San Francisco,1
7,8,2021-02-14 00:39:40,NaN,6,San Francisco,0
10,11,2021-02-15 11:23:45,NaN,6,San Francisco,1
3,4,2021-04-22 01:22:34,NaN,7,Paris,0
24,25,2021-02-19 13:01:37,NaN,7,Busan,1
19,20,2021-01-16 03:24:06,NaN,7,San Deigo,1
22,23,2021-03-02 04:07:27,NaN,13,San Francisco,1
28,29,2021-03-21 08:33:48,NaN,13,Los Angeles,1
8,9,2021-03-25 22:14:04,NaN,13,San Francisco,1
6,7,2021-02-22 15:31:42,NaN,13,Boston,1
"""

data = io.StringIO(datastring)
dddd = pd.read_csv(data, sep=",").reset_index()


dddd

,index,ride_id,start_time,end_time,driver_id,region,is_completed
0,4,5,2021-01-15 21:02:58,NaN,2,Cape Town,1
1,26,27,2021-03-31 21:51:00,NaN,2,San Francisco,1
2,0,1,2021-04-07 10:31:41,NaN,4,San Francisco,1
3,23,24,2021-02-20 06:37:07,NaN,4,San Francisco,1
4,7,8,2021-02-14 00:39:40,NaN,6,San Francisco,0
5,10,11,2021-02-15 11:23:45,NaN,6,San Francisco,1
6,3,4,2021-04-22 01:22:34,NaN,7,Paris,0
7,24,25,2021-02-19 13:01:37,NaN,7,Busan,1
8,19,20,2021-01-16 03:24:06,NaN,7,San Deigo,1
9,22,23,2021-03-02 04:07:27,NaN,13,San Francisco,1


In [325]:
dddd.merge(dddd.groupby('driver_id').region.first().to_frame().reset_index(), on='driver_id', how='right').drop('region_x', axis=1).rename(columns={'region_y':'region'})

,index,ride_id,start_time,end_time,driver_id,is_completed,region
0,4,5,2021-01-15 21:02:58,NaN,2,1,Cape Town
1,26,27,2021-03-31 21:51:00,NaN,2,1,Cape Town
2,0,1,2021-04-07 10:31:41,NaN,4,1,San Francisco
3,23,24,2021-02-20 06:37:07,NaN,4,1,San Francisco
4,7,8,2021-02-14 00:39:40,NaN,6,0,San Francisco
5,10,11,2021-02-15 11:23:45,NaN,6,1,San Francisco
6,3,4,2021-04-22 01:22:34,NaN,7,0,Paris
7,24,25,2021-02-19 13:01:37,NaN,7,1,Paris
8,19,20,2021-01-16 03:24:06,NaN,7,1,Paris
9,22,23,2021-03-02 04:07:27,NaN,13,1,San Francisco


In [242]:
dddd

,index,ride_id,start_time,end_time,driver_id,is_completed,region
0,4,5,2021-01-15 21:02:58,NaN,2,1,Cape Town
1,26,27,2021-03-31 21:51:00,NaN,2,1,Cape Town
2,0,1,2021-04-07 10:31:41,NaN,4,1,San Francisco
3,23,24,2021-02-20 06:37:07,NaN,4,1,San Francisco
4,7,8,2021-02-14 00:39:40,NaN,6,0,San Francisco
5,10,11,2021-02-15 11:23:45,NaN,6,1,San Francisco
6,3,4,2021-04-22 01:22:34,NaN,7,0,Paris
7,24,25,2021-02-19 13:01:37,NaN,7,1,Paris
8,19,20,2021-01-16 03:24:06,NaN,7,1,Paris
9,22,23,2021-03-02 04:07:27,NaN,13,1,San Francisco


In [244]:
d="""type,boq a,boq b,progress a,progress b
P,Total point,NaN,Terminated,NaN
Total,2,2,1,2
Q,Total point,NaN,Terminated,NaN
Total,1,1,3,3
R,Total point,NaN,Terminated,NaN
Total,4,2,5,1
S,Total point,NaN,Terminated,NaN
Total,4,5,2,1"""
df = gdf(d)
df

,type,boq a,boq b,progress a,progress b
0,P,Total point,NaN,Terminated,NaN
1,Total,2,2.0,1,2.0
2,Q,Total point,NaN,Terminated,NaN
3,Total,1,1.0,3,3.0
4,R,Total point,NaN,Terminated,NaN
5,Total,4,2.0,5,1.0
6,S,Total point,NaN,Terminated,NaN
7,Total,4,5.0,2,1.0


In [246]:
def backfill(df):
    bfill_list = ['Total point', np.nan, 'Terminated']
    if df.columns.contains(bfill_list):
        df.columns.bfill

In [248]:
#np.column_stack([df[col].str.contains(r"\^", na=False) for col in df])

In [267]:
bfill_list = ['Total point', 'Terminated']
df[df.apply(lambda row: row.astype(str).str.contains('|'.join(bfill_list)), axis=1)]

,type,boq a,boq b,progress a,progress b
0,NaN,Total point,NaN,Terminated,NaN
1,NaN,NaN,NaN,NaN,NaN
2,NaN,Total point,NaN,Terminated,NaN
3,NaN,NaN,NaN,NaN,NaN
4,NaN,Total point,NaN,Terminated,NaN
5,NaN,NaN,NaN,NaN,NaN
6,NaN,Total point,NaN,Terminated,NaN
7,NaN,NaN,NaN,NaN,NaN


In [272]:
d = """A,B
W,1
WT,2"""
df = gdf(d)
df

,A,B
0,W,1
1,WT,2


In [270]:
df['C']

0    2
1    4
Name: C, dtype: object

In [299]:
d = """date,Device_Id,value
28-12-2018 00:00,d1,0.014
28-12-2018 00:15,d1,0.012
28-12-2018 00:30,d1,0.012
28-12-2018 00:45,d1,0.014
28-12-2018 01:00,d1,0.012
28-12-2018 01:15,d1,0.012
28-12-2018 01:30,d1,0.012
28-12-2018 01:45,d1,0.012
28-12-2018 02:00,d1,0.014
28-12-2018 02:15,d1,0.012"""
df = gdf(d)
df['date'] = pd.to_datetime(df.date)
df

,date,Device_Id,value
0,2018-12-28 00:00:00,d1,0.014
1,2018-12-28 00:15:00,d1,0.012
2,2018-12-28 00:30:00,d1,0.012
3,2018-12-28 00:45:00,d1,0.014
4,2018-12-28 01:00:00,d1,0.012
5,2018-12-28 01:15:00,d1,0.012
6,2018-12-28 01:30:00,d1,0.012
7,2018-12-28 01:45:00,d1,0.012
8,2018-12-28 02:00:00,d1,0.014
9,2018-12-28 02:15:00,d1,0.012


In [300]:
df.set_index('date', inplace=True)

In [297]:
df.where(df.Device_Id.nq('d1'), df.index.between('28-12-2018 01:00:00', '28-12-2018 06:00:00'))

AttributeError: 'DatetimeIndex' object has no attribute 'between'

In [302]:
df = pd.DataFrame({ 'date':['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04'],
                    'Sam.BP':[12,11,35,48], 
                    'Sam.HR':[12,11,35,48],
                    'John.BP':[22,41,35,48], 
                    'John.HR':[32,51,35,48],
                    'UG.BP':[22,21,25,28], 
                    'UG.HR':[32,31,35,38]})
df

,date,Sam.BP,Sam.HR,John.BP,John.HR,UG.BP,UG.HR
0,2020-01-01,12,12,22,32,22,32
1,2020-01-02,11,11,41,51,21,31
2,2020-01-03,35,35,35,35,25,35
3,2020-01-04,48,48,48,48,28,38


In [304]:
df_expected = pd.DataFrame({ 'date':['2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04', '2020-01-01', '2020-01-02', '2020-01-03', '2020-01-04'],
                    'name': ['Sam', 'Sam', 'Sam', 'Sam', 'John', 'John', 'John', 'John', 'UG', 'UG', 'UG', 'UG'],
                    'BP':[12,11,35,48, 22,41,35,48, 22,21,25,28], 
                    'HR':[12,11,35,48, 32,51,35,48, 32,31,35,38]})
df_expected

,date,name,BP,HR
0,2020-01-01,Sam,12,12
1,2020-01-02,Sam,11,11
2,2020-01-03,Sam,35,35
3,2020-01-04,Sam,48,48
4,2020-01-01,John,22,32
5,2020-01-02,John,41,51
6,2020-01-03,John,35,35
7,2020-01-04,John,48,48
8,2020-01-01,UG,22,32
9,2020-01-02,UG,21,31


In [308]:
dddd

,index,ride_id,start_time,end_time,driver_id,region,is_completed
0,4,5,2021-01-15 21:02:58,NaN,2,Cape Town,1
1,26,27,2021-03-31 21:51:00,NaN,2,San Francisco,1
2,0,1,2021-04-07 10:31:41,NaN,4,San Francisco,1
3,23,24,2021-02-20 06:37:07,NaN,4,San Francisco,1
4,7,8,2021-02-14 00:39:40,NaN,6,San Francisco,0
5,10,11,2021-02-15 11:23:45,NaN,6,San Francisco,1
6,3,4,2021-04-22 01:22:34,NaN,7,Paris,0
7,24,25,2021-02-19 13:01:37,NaN,7,Busan,1
8,19,20,2021-01-16 03:24:06,NaN,7,San Deigo,1
9,22,23,2021-03-02 04:07:27,NaN,13,San Francisco,1


In [11]:
d = """id,tag_positions
1,center
2,right
3,['left']
4,['center']
5,['left']
6,['right']
7,left"""
df = gdf(d)
df

,id,tag_positions
0,1,center
1,2,right
2,3,['left']
3,4,['center']
4,5,['left']
5,6,['right']
6,7,left


In [12]:
df = df.explode('tag_positions')
df

,id,tag_positions
0,1,center
1,2,right
2,3,['left']
3,4,['center']
4,5,['left']
5,6,['right']
6,7,left


In [82]:
tuples = list(zip(*[
    ["a","a","b","b","c","c"],
    ["product","price","product","price","product","price"],
]))
columns = pd.MultiIndex.from_tuples(tuples)
df = pd.DataFrame(
    [['apple', 1, 'oranges', 2, 'bananas', 1],
    ['oranges', 4, 'bananas', 3, 'oranges', 2],
    ['bananas', 5, 'apple', 4, 'apple', 3]],
columns=columns)
df

a              b              c      
   product price  product price  product price
0    apple     1  oranges     2  bananas     1
1  oranges     4  bananas     3  oranges     2
2  bananas     5    apple     4    apple     3

In [83]:
df = df.stack(0).reset_index(1).pivot(
    columns='level_1', index='product').reset_index()
df.columns = df.columns.droplevel(0).rename(None)
df

,,a,b,c
0,apple,1,4,3
1,bananas,5,3,1
2,oranges,4,2,2


In [124]:
import pandas as pd
df = pd.DataFrame()
df['filename'] = ['118_3.JPG', '118_3.JPG', '118_3.JPG', '118_3.JPG', '118_3.JPG', '15_7.JPG', '15_7.JPG', '15_7.JPG', '15_7.JPG', '15_7.JPG','203_4.JPG', '203_4.JPG', '203_4.JPG', '203_4.JPG', '203_4.JPG']
df['cvxh_len'] = [100, 200, 3000, 2800, 29, 200, 400, 2, 1, 0, 5000, 6000, 9000, 11000, 15000]
df['date'] = ["2018-12-14", "2018-12-15", "2018-12-16", "2018-12-17", "2018-12-18", "2018-12-14", "2018-12-15", "2018-12-16", "2018-12-17", "2018-12-18", "2018-12-14", "2018-12-15", "2018-12-16", "2018-12-17", "2018-12-18" ]
df["date"] = pd.to_datetime(df["date"])
df

,filename,cvxh_len,date
0,118_3.JPG,100,2018-12-14
1,118_3.JPG,200,2018-12-15
2,118_3.JPG,3000,2018-12-16
3,118_3.JPG,2800,2018-12-17
4,118_3.JPG,29,2018-12-18
5,15_7.JPG,200,2018-12-14
6,15_7.JPG,400,2018-12-15
7,15_7.JPG,2,2018-12-16
8,15_7.JPG,1,2018-12-17
9,15_7.JPG,0,2018-12-18


In [89]:
df.loc[df["cvxh_len"]  > df["cvxh_len"].shift()]

,filename,cvxh_len,date
3,118_3.JPG,2800,2018-12-17
4,118_3.JPG,29,2018-12-18
7,15_7.JPG,2,2018-12-16
8,15_7.JPG,1,2018-12-17
9,15_7.JPG,0,2018-12-18


In [90]:
df.groupby((df["cvxh_len"] > df["cvxh_len"].shift()).cumsum().values).first()

,filename,cvxh_len,date
0,118_3.JPG,100,2018-12-14
1,118_3.JPG,200,2018-12-15
2,118_3.JPG,3000,2018-12-16
3,15_7.JPG,200,2018-12-14
4,15_7.JPG,400,2018-12-15
5,203_4.JPG,5000,2018-12-14
6,203_4.JPG,6000,2018-12-15
7,203_4.JPG,9000,2018-12-16
8,203_4.JPG,11000,2018-12-17
9,203_4.JPG,15000,2018-12-18


In [125]:
df.groupby(['filename', 'date']).transform(lambda x:(x["cvxh_len"] > x["cvxh_len"].shift()))

KeyError: 'cvxh_len'

In [119]:
d = """Description
ABC12345679 132465
Test ABC12346548
Test ABC1231321 4645"""
data = gdf(d)
data

,Description
0,ABC12345679 132465
1,Test ABC12346548
2,Test ABC1231321 4645


In [120]:
data['extract'] = data.Description.str.extract(r'([A-C]+[0-9]+)')
data

,Description,extract
0,ABC12345679 132465,ABC12345679
1,Test ABC12346548,ABC12346548
2,Test ABC1231321 4645,ABC1231321


In [189]:
df = pd.DataFrame({
    'col1': [1,2,3,4,5,5,4,3,2,1],
    'col2':[6,7,8,9,10,10,9,8,7,6],
    'col3':['a','b','c','d','e','a','b','c','d','e']
})
df = df.pivot_table(
    'col2', columns='col3', aggfunc=(lambda x:x.to_list())
).apply(pd.Series.explode).rename_axis(None, axis=1).reset_index()
df

,index,a,b,c,d,e
0,col2,6,7,8,9,10
1,col2,10,9,8,7,6


In [190]:
uni3pa = {'3PA SMARTPAY': ['[Item].[Item].[10006543 - Smartpay User Licence]',
'[Item].[Item].[10006544 - SmartPay User Licence per Site]'],
'3PA OTHER': ['[Item].[Item].[10001234 - 3rd Party App User Licence]', 
'[Item].[Item].[10001235 - 3rd Party App User Licence (min 3 per site]',
'[Item].[Item].[10001236 - 3rd Party Apps Single User]']}
uni3pa

{'3PA SMARTPAY': ['[Item].[Item].[10006543 - Smartpay User Licence]',
  '[Item].[Item].[10006544 - SmartPay User Licence per Site]'],
 '3PA OTHER': ['[Item].[Item].[10001234 - 3rd Party App User Licence]',
  '[Item].[Item].[10001235 - 3rd Party App User Licence (min 3 per site]',
  '[Item].[Item].[10001236 - 3rd Party Apps Single User]']}

In [191]:
{k: ' + '.join(v) for k, v in uni3pa.items()}

{'3PA SMARTPAY': '[Item].[Item].[10006543 - Smartpay User Licence] + [Item].[Item].[10006544 - SmartPay User Licence per Site]',
 '3PA OTHER': '[Item].[Item].[10001234 - 3rd Party App User Licence] + [Item].[Item].[10001235 - 3rd Party App User Licence (min 3 per site] + [Item].[Item].[10001236 - 3rd Party Apps Single User]'}

In [196]:
d = {'a':'abc', 'b':'bcd'}
{k: '/xyx'+ '/' +d[k] for k, v in d.items()}

{'a': '/xyx/abc', 'b': '/xyx/bcd'}

In [197]:
units_col = pd.Series([
'mcg',
'ml',
'usec',
'kg',
'hour',
'mol'
])
units_col

0     mcg
1      ml
2    usec
3      kg
4    hour
5     mol
dtype: object

In [199]:
split_units = pd.Series([
['mc','g'],
['m','l'],
['u','sec'],
['k','g'],
['hour'], 
['mol'] 
])
split_units

0     [mc, g]
1      [m, l]
2    [u, sec]
3      [k, g]
4      [hour]
5       [mol]
dtype: object

In [291]:
d = """date,ent_id,val
2021-03-23,109,61
2021-03-12,104,64
2021-03-31,101,61
2021-03-30,103,64
2021-04-01,111,32
2021-04-01,153,39
2021-04-30,101,51
2021-04-30,103,53
2021-05-12,101,28
2021-05-07,103,26
2021-05-05,171,47
2021-05-05,183,61
2021-06-06,131,45
2021-06-06,133,78
2021-06-30,101,23
2021-06-30,103,31"""
df = gdf(d)
df
df.groupby(df['date'].dt.to_period('M'))['date'].transform('max')

AttributeError: Can only use .dt accessor with datetimelike values

In [224]:
df['date'] = pd.to_datetime(df['date'], format='%Y-%m-%d')
df['max_avl_d'] =  df.date.dt.to_period('M').dt.to_timestamp('M')
df

,date,ent_id,val,max_avl_d
0,2021-03-23,109,61,2021-03-31
1,2021-03-12,104,64,2021-03-31
2,2021-03-31,101,61,2021-03-31
3,2021-03-30,103,64,2021-03-31
4,2021-04-01,111,32,2021-04-30
5,2021-04-01,153,39,2021-04-30
6,2021-04-30,101,51,2021-04-30
7,2021-04-30,103,53,2021-04-30
8,2021-05-12,101,28,2021-05-31
9,2021-05-07,103,26,2021-05-31


In [236]:
df = pd.DataFrame({'date':['2020 Q1','2020 Q2','2020 Q3','2020 Q4'],
                  'value':['5,268','6,166', '7,085', '6,748']})
#df['date'] = df.date.str.split(' ').str[0]
#df['date'] = pd.to_datetime(df['date'])
#df.set_index(['date'],inplace=True)
df

,date,value
0,2020 Q1,"5,268"
1,2020 Q2,"6,166"
2,2020 Q3,"7,085"
3,2020 Q4,"6,748"


In [234]:
Y_df = df.resample('Y').last()
Y_df

,value
date,
2020-12-31,"6,748"


In [243]:
df['date'] = df['date'].str.replace(r'(\d+) (Q\d)', r'\1-\2')
df['date'] = pd.PeriodIndex(df.date, freq='Q').to_timestamp()
df

<ipython-input-243-2bd5b28c7310>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['date'] = df['date'].str.replace(r'(\d+) (Q\d)', r'\1-\2')


,date,value
0,2020-01-01,"5,268"
1,2020-04-01,"6,166"
2,2020-07-01,"7,085"
3,2020-10-01,"6,748"


In [289]:
d="""TransactionId,StartTime,EndTime,Power
xyza123,2018.01.01 07:07:34,2018.01.01 07:34:08,70
hjker383,2018.01.01 10:21:00,2018.01.01 11:40:08,23"""
df = gdf(d)
df['StartTime'] = pd.to_datetime(df.StartTime)
df['EndTime'] = pd.to_datetime(df.EndTime)


In [290]:
df['StartTime'] = pd.to_datetime(df.StartTime)
df['EndTime'] = pd.to_datetime(df.EndTime)
df = df.resample('15min', on='StartTime').first().dropna().rename_axis('New Starttime').reset_index()
df = df.resample('15min', on='EndTime').first().dropna().rename_axis('New EndTime').reset_index()


,New EndTime,New Starttime,TransactionId,StartTime,EndTime,Power
0,2018-01-01 07:30:00,2018-01-01 07:00:00,xyza123,2018-01-01 07:07:34,2018-01-01 07:34:08,70.0
1,2018-01-01 11:30:00,2018-01-01 10:15:00,hjker383,2018-01-01 10:21:00,2018-01-01 11:40:08,23.0


In [318]:
df = pd.DataFrame({'id':['x1', 'x1', 'x1', 'x1', 'x2', 'x2', 'x2', 'x2']
               ,  'date':['2021-01-01','2021-01-02',
               '2021-01-03','2021-01-04','2021-01-05',
               '2021-01-06','2021-01-07','2021-01-08'],
           'revenue':[5,3,2,
                      10,12,2,
                      1,0]})
df["date"] = pd.to_datetime(df['date'])
df = df.groupby('id').resample('W', on='date').sum().reset_index()
df['date'] = df.date - pd.Timedelta('6D')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3 entries, 0 to 2
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   id       3 non-null      object        
 1   date     3 non-null      datetime64[ns]
 2   revenue  3 non-null      int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 200.0+ bytes


In [300]:
d = """Distance,Value,Distance.1,Value.1
-15,1,-15,2
-14.9,3,-14.995,4
-14.8,4,-14.992,2"""
df = gdf(d)
df

,Distance,Value,Distance.1,Value.1
0,-15.0,1,-15.000,2
1,-14.9,3,-14.995,4
2,-14.8,4,-14.992,2


In [305]:
df0 = pd.DataFrame(
        columns=["a", "b", "key"],
        data=[{"a": 4, "b": 5, "key": 1}, {"a": 3, "b": 9, "key": 2}],
    )
df0.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   a       2 non-null      int64
 1   b       2 non-null      int64
 2   key     2 non-null      int64
dtypes: int64(3)
memory usage: 176.0 bytes


In [328]:
df1 = pd.DataFrame({'a':[4,3], 'b':[None,None], 'key':[3,9]}, 
                   dtype=int)
#df1['b'] = df1.b.astype('datetime64[ns]').astype(int)
df1['b'] = df1.b.astype('string').astype(np.int16)
df1.info()



TypeError: int() argument must be a string, a bytes-like object or a number, not 'NAType'

In [332]:
df = pd.read_csv('te.csv', header=None)
df

,0,1,2,3,4,5,6
0,5,1730572382,8236,1600956334,12.95,1012.38,64.80
1,6,1730551647,8266,1600956334,13.42,1012.43,64.66
2,7,1730582880,8290,1600956334,13.18,1012.60,61.39


In [337]:
[df.to_string(header=False,
                  index=False,
                  index_names=False)].to_csv()

['5 1730572382 8236 1600956334 12.95 1012.38 64.80\n6 1730551647 8266 1600956334 13.42 1012.43 64.66\n7 1730582880 8290 1600956334 13.18 1012.60 61.39']

In [339]:
d = """index,date
A,1.1.20
A,1.2.20
B,1.1.20
B,5.5.21"""

df = gdf(d)
df['date'] = pd.to_datetime(df.date)
df

,index,date
0,A,2020-01-01
1,A,2020-01-02
2,B,2020-01-01
3,B,2021-05-05


In [350]:
df['bool'] = np.where(df.groupby(['index', pd.Grouper(key='date', freq='Y')])['index'].transform('count') >= 2, True, False) 
df

,index,date,bool
0,A,2020-01-01,True
1,A,2020-01-02,True
2,B,2020-01-01,False
3,B,2021-05-05,False


In [354]:
d = """index,A,B,C,D,E
0,92,84,,,
1,98,49,,,
2,49,68,,,
3,0,58,,,
4,91,95,,,
5,47,56,52,25,58
6,86,71,34,39,40
7,80,78,0,86,12
8,0,8,30,88,42
9,69,83,7,65,60
10,93,39,10,90,45"""
df1 = gdf(d)
df1

,index,A,B,C,D,E
0,0,92,84,NaN,NaN,NaN
1,1,98,49,NaN,NaN,NaN
2,2,49,68,NaN,NaN,NaN
3,3,0,58,NaN,NaN,NaN
4,4,91,95,NaN,NaN,NaN
5,5,47,56,52.0,25.0,58.0
6,6,86,71,34.0,39.0,40.0
7,7,80,78,0.0,86.0,12.0
8,8,0,8,30.0,88.0,42.0
9,9,69,83,7.0,65.0,60.0


In [355]:
d = """index,C,D,E,F
0,27,95,51,45
1,99,33,92,67
2,68,37,29,65
3,99,25,48,40
4,33,74,55,66
13,65,76,19,62"""
df2 = gdf(d)
df2

,index,C,D,E,F
0,0,27,95,51,45
1,1,99,33,92,67
2,2,68,37,29,65
3,3,99,25,48,40
4,4,33,74,55,66
5,13,65,76,19,62


In [356]:
df1 = df1.rename(columns={'index':'match'})
df2 = df2.rename(columns={'index':'match'})

In [364]:
pd.merge(df1, df2, on='match', how='left')

,match,A,B,C_x,D_x,E_x,C_y,D_y,E_y,F
0,0,92,84,NaN,NaN,NaN,27.0,95.0,51.0,45.0
1,1,98,49,NaN,NaN,NaN,99.0,33.0,92.0,67.0
2,2,49,68,NaN,NaN,NaN,68.0,37.0,29.0,65.0
3,3,0,58,NaN,NaN,NaN,99.0,25.0,48.0,40.0
4,4,91,95,NaN,NaN,NaN,33.0,74.0,55.0,66.0
5,5,47,56,52.0,25.0,58.0,NaN,NaN,NaN,NaN
6,6,86,71,34.0,39.0,40.0,NaN,NaN,NaN,NaN
7,7,80,78,0.0,86.0,12.0,NaN,NaN,NaN,NaN
8,8,0,8,30.0,88.0,42.0,NaN,NaN,NaN,NaN
9,9,69,83,7.0,65.0,60.0,NaN,NaN,NaN,NaN


In [3]:
d = """index,date
A,1.1.20
A,1.2.20
B,1.1.20
B,5.5.21
C,1.1.20
C,10.1.20
D,1.1.20
D,10.1.20
D,10.5.20"""
df = gdf(d)
df['date'] = pd.to_datetime(df.date, dayfirst=True)

,index,date
0,A,1.1.20
1,A,1.2.20
2,B,1.1.20
3,B,5.5.21
4,C,1.1.20
5,C,10.1.20
6,D,1.1.20
7,D,10.1.20
8,D,10.5.20


In [4]:
df['date'] = pd.to_datetime(df.date, dayfirst=True)
df

,index,date
0,A,2020-01-01
1,A,2020-02-01
2,B,2020-01-01
3,B,2021-05-05
4,C,2020-01-01
5,C,2020-01-10
6,D,2020-01-01
7,D,2020-01-10
8,D,2020-05-10


In [493]:
np.where(~(df.groupby(['index', pd.Grouper(key='date', freq='Y'),
    pd.Grouper(key='date', freq='M')]).index.transform('count') >= 2) &
((df.groupby(['index',
    pd.Grouper(key='date', freq='M')]).index.transform('count') >1)),
True, False)

array([False, False, False, False, False, False, False, False, False])

In [23]:
~(df.groupby(['index', pd.Grouper(key='date', freq='Y'),
    pd.Grouper(key='date', freq='M'), ]).index.transform('count') >= 2)

0     True
1     True
2     True
3     True
4    False
5    False
6    False
7    False
8     True
Name: index, dtype: bool

In [25]:
conditions = [
    ((df.groupby(['index', pd.Grouper(key='date', freq='Y'), 
    pd.Grouper(key='date', freq='M')]).index.transform('count') >= 2)),
    (df.groupby(['index', pd.Grouper(key='date', freq='Y')]).index.transform('count') >= 3) &
    (df.groupby(['index', pd.Grouper(key='date', freq='M')]).index.transform('count') >= 2),
]
choices = [False, True]
np.select(conditions, choices)


array([0, 0, 0, 0, 0, 0, 0, 0, 0])

In [29]:
~(df.groupby(['index', pd.Grouper(key='date', freq='Y'), 
    pd.Grouper(key='date', freq='M')]).index.transform('count') >= 2)

0     True
1     True
2     True
3     True
4    False
5    False
6    False
7    False
8     True
Name: index, dtype: bool

In [48]:
df['bool'] = ((df.groupby(['index', pd.Grouper(key='date', freq='Y')]).index.transform('count') >= 2)& (df.groupby(['index', pd.Grouper(key='date', freq='M')]).index.transform('count') != 2)) | ((df.groupby(['index', pd.Grouper(key='date', freq='Y')]).index.transform('count') >= 3) & (df.groupby(['index', pd.Grouper(key='date', freq='M')]).index.transform('count') >= 2))

In [49]:
df

,index,date,bool
0,A,2020-01-01,True
1,A,2020-02-01,True
2,B,2020-01-01,False
3,B,2021-05-05,False
4,C,2020-01-01,False
5,C,2020-01-10,False
6,D,2020-01-01,True
7,D,2020-01-10,True
8,D,2020-05-10,True


In [64]:
d = """Nr,DatumZeit,Erdtemp,Heizung
0,25.04.21 12:58:42,21.8,1
1,25.04.21 12:58:54,21.8,1
2,25.04.21 12:59:06,21.9,1
3,25.04.21 12:59:18,21.9,1
4,25.04.21 12:59:29,21.9,1
5,25.04.21 12:59:41,22.0,1
6,25.04.21 12:59:53,22.0,1
7,25.04.21 13:00:05,22.1,1
8,25.04.21 13:00:16,22.1,0
9,25.04.21 13:00:28,22.1,0
10,25.04.21 13:00:40,22.1,0
11,25.04.21 13:00:52,22.2,0
12,25.04.21 13:01:03,22.2,0
13,25.04.21 13:01:15,22.2,1
14,25.04.21 13:01:27,22.2,1
15,25.04.21 13:01:39,22.3,1
16,25.04.21 13:01:50,22.3,1
17,25.04.21 13:02:02,22.4,1
18,25.04.21 13:02:14,22.4,1
19,25.04.21 13:02:26,22.4,0
20,25.04.21 13:02:37,22.4,1
21,25.04.21 13:02:49,22.4,0
22,25.04.21 13:03:01,22.4,0
23,25.04.21 13:03:13,22.5,0
24,25.04.21 13:03:25,22.4,0"""
Tab1= gdf(d)
Tab1['DatumZeit'] = pd.to_datetime(Tab1.DatumZeit)
#Tab1 = Tab1.set_index('DatumZeit')
#Tab1.resample('D').Heizung.sum()
Tab1['DatumZeit'] = pd.to_datetime(Tab1.DatumZeit)
Tab1 = Tab1.groupby(pd.Grouper(key='DatumZeit', freq='D')).Heizung.sum().to_frame()
Tab1

,Heizung
DatumZeit,
2021-04-25,15


In [55]:
df.resample('D').Heizung.sum()

DatumZeit
2021-04-25    15
Freq: D, Name: Heizung, dtype: int64

In [3]:
d = """Entity,PL/BS,RSX Account,C Pack Account,G/L Account,PC,Profit Center Descr.,Trading partner,Trading partner Descr.,1,2,3,4
6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,#,Not assigned,366,328,5,481
6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R3020,DACH,984,659,475,894
6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R6001,SA,577,472,687,887
6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R6079,BX,416,400,159,432
6156,BS,RSX4000,400100,400100D,X6156999,Bal. Sheet-6156,R6154,INC,270,741,317,564
6157,BS,RSX4000,400100,400100D,X6157999,Bal. Sheet-6157,#,Not assigned,44,437,200,247
6157,BS,RSX4000,400100,400100D,X6157999,Bal. Sheet-6157,R6267,France,597,612,87,48
6157,BS,RSX4000,400100,400100S,X6157999,Bal. Sheet-6157,R6154,INC,844,251,800,624
6154,PL,RSX6240,640100,6408900,X6154999,Others,R6156,AUS,592,638,945,43
6156,PL,RSX6240,656000,6560000,X6156999,Other,R6157,UK,613,26,462,224
6156,PL,RSX6240,656100,6561000,X6156901,Executive & Admin,R6157,UK,777,131,579,250
6227,PL,RSX6240,642000,6452250,X6227009,Platform-Brand Partn,R6154,INC,102,486,356,483"""
df = gdf(d)
df

,Entity,PL/BS,RSX Account,C Pack Account,G/L Account,PC,Profit Center Descr.,Trading partner,Trading partner Descr.,1,2,3,4
0,6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,#,Not assigned,366,328,5,481
1,6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R3020,DACH,984,659,475,894
2,6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R6001,SA,577,472,687,887
3,6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R6079,BX,416,400,159,432
4,6156,BS,RSX4000,400100,400100D,X6156999,Bal. Sheet-6156,R6154,INC,270,741,317,564
5,6157,BS,RSX4000,400100,400100D,X6157999,Bal. Sheet-6157,#,Not assigned,44,437,200,247
6,6157,BS,RSX4000,400100,400100D,X6157999,Bal. Sheet-6157,R6267,France,597,612,87,48
7,6157,BS,RSX4000,400100,400100S,X6157999,Bal. Sheet-6157,R6154,INC,844,251,800,624
8,6154,PL,RSX6240,640100,6408900,X6154999,Others,R6156,AUS,592,638,945,43
9,6156,PL,RSX6240,656000,6560000,X6156999,Other,R6157,UK,613,26,462,224


In [5]:
df = pd.DataFrame({'year': {0: 1976, 1: 1976, 2: 1976, 3: 1976, 4: 1976},
                   'state': {0: 'ALABAMA', 1: 'ALABAMA', 2: 'ALABAMA',
                             3: 'ALABAMA', 4: 'ALABAMA'},
                   'candidate': {0: 'CARTER, JIMMY', 1: 'FORD, GERALD',
                                 2: 'MADDOX, LESTER', 3: 'BUBAR, BENJAMIN',
                                 4: 'HALL, GUS'},
                   'percvotes': {0: 55.727269, 1: 42.614871, 2: 0.777613, 3: 0.563808,
                                 4: 0.165194},
                   'electoral_votes': {0: 9, 1: 9, 2: 9, 3: 9, 4: 9},
                   'perc_evotes': {0: 5.015454, 1: 3.835338, 2: 0.069985,
                                   3: 0.050743, 4: 0.014867},
                   'vote_frac': {0: 0.015454, 1: 0.835338, 2: 0.069985,
                                 3: 0.050743, 4: 0.014867},
                   'vote_int': {0: 5, 1: 3, 2: 0, 3: 0, 4: 0}})
df

,year,state,candidate,percvotes,electoral_votes,perc_evotes,vote_frac,vote_int
0,1976,ALABAMA,"CARTER, JIMMY",55.727269,9,5.015454,0.015454,5
1,1976,ALABAMA,"FORD, GERALD",42.614871,9,3.835338,0.835338,3
2,1976,ALABAMA,"MADDOX, LESTER",0.777613,9,0.069985,0.069985,0
3,1976,ALABAMA,"BUBAR, BENJAMIN",0.563808,9,0.050743,0.050743,0
4,1976,ALABAMA,"HALL, GUS",0.165194,9,0.014867,0.014867,0


In [9]:
def apply_extra_e_votes(grp):
    # Get First Electoral Vote
    # (Assumes first row in group contains the
    #  correct number of electoral votes for the group)
    available_e_votes = grp['electoral_votes'].iloc[0]
    # Get the Sum of the vote_int column
    current_e_votes = grp['vote_int'].sum()
    # If there more available votes than votes cast
    if available_e_votes > current_e_votes:
        # Update the 'vote_int' column at the max value of 'vote_frac'
        grp.loc[
            grp['vote_frac'].idxmax(),
            'vote_int'
        ] += available_e_votes - current_e_votes  # (Remaining Votes)
    return grp


# Groupby and Apply Function
new_df = df.groupby(['year', 'state']).apply(apply_extra_e_votes)

new_df

,year,state,candidate,percvotes,electoral_votes,perc_evotes,vote_frac,vote_int
0,1976,ALABAMA,"CARTER, JIMMY",55.727269,9,5.015454,0.015454,5
1,1976,ALABAMA,"FORD, GERALD",42.614871,9,3.835338,0.835338,4
2,1976,ALABAMA,"MADDOX, LESTER",0.777613,9,0.069985,0.069985,0
3,1976,ALABAMA,"BUBAR, BENJAMIN",0.563808,9,0.050743,0.050743,0
4,1976,ALABAMA,"HALL, GUS",0.165194,9,0.014867,0.014867,0


In [10]:
df = pd.DataFrame(
    [
        [np.nan, np.nan, 1, 1],
        [1, np.nan, np.nan, np.nan],
        [np.nan, 1, np.nan, 1],
        [1, 1, np.nan, 1]
    ],
    columns=['A', 'B', 'C', 'D']
)
df

,A,B,C,D
0,NaN,NaN,1.0,1.0
1,1.0,NaN,NaN,NaN
2,NaN,1.0,NaN,1.0
3,1.0,1.0,NaN,1.0


In [15]:
df.ffill().get_dummies()

AttributeError: 'DataFrame' object has no attribute 'get_dummies'

In [16]:
dic = {'Text': ['i8am going to school', 'i8am a very good boy']}
df = pd.DataFrame(dic)
d = df['Text'].replace(to_replace='i8am', value='i am')
d

0    i8am going to school
1    i8am a very good boy
Name: Text, dtype: object

In [17]:
dic = {'Text': ['i8am going to school', 'i8am a very good boy']}
df = pd.DataFrame(dic)
df

,Text
0,i8am going to school
1,i8am a very good boy


In [20]:
d = df.Text.str.replace('i8am', 'I am')
d

0    I am going to school
1    I am a very good boy
Name: Text, dtype: object

In [31]:
d = """index,date
A,1.1.20
A,1.2.20
B,1.1.20
B,5.5.21
C,1.1.20
C,10.1.20
D,1.1.20
D,10.1.20
D,10.5.20"""
df = gdf(d)
df['date'] = pd.to_datetime(df.date, dayfirst=True)
df

,index,date
0,A,2020-01-01
1,A,2020-02-01
2,B,2020-01-01
3,B,2021-05-05
4,C,2020-01-01
5,C,2020-01-10
6,D,2020-01-01
7,D,2020-01-10
8,D,2020-05-10


In [34]:
df['date'] = pd.to_datetime(df.date, dayfirst=True)

def resampled_data(freq):
    return df.groupby(['index', pd.Grouper(key='date', freq=freq)]).index.transform('count')

yg = resampled_data('Y')
mg = resampled_data('M')

df['bool'] = ((yg >= 2) & (mg != 2)) | ((yg >= 3) & (mg >=2))
df

,index,date,bool
0,A,2020-01-01,True
1,A,2020-02-01,True
2,B,2020-01-01,False
3,B,2021-05-05,False
4,C,2020-01-01,False
5,C,2020-01-10,False
6,D,2020-01-01,True
7,D,2020-01-10,True
8,D,2020-05-10,True


In [35]:
d = """Entity,PL/BS,RSX Account,C Pack Account,G/L Account,PC,Profit Center Descr.,Trading partner,Trading partner Descr.,1,2,3,4
6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,#,Not assigned,366,328,5,481
6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R3020,DACH,984,659,475,894
6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R6001,SA,577,472,687,887
6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R6079,BX,416,400,159,432
6156,BS,RSX4000,400100,400100D,X6156999,Bal. Sheet-6156,R6154,INC,270,741,317,564
6157,BS,RSX4000,400100,400100D,X6157999,Bal. Sheet-6157,#,Not assigned,44,437,200,247
6157,BS,RSX4000,400100,400100D,X6157999,Bal. Sheet-6157,R6267,France,597,612,87,48
6157,BS,RSX4000,400100,400100S,X6157999,Bal. Sheet-6157,R6154,INC,844,251,800,624
6154,PL,RSX6240,640100,6408900,X6154999,Others,R6156,AUS,592,638,945,43
6156,PL,RSX6240,656000,6560000,X6156999,Other,R6157,UK,613,26,462,224
6156,PL,RSX6240,656100,6561000,X6156901,Executive & Admin,R6157,UK,777,131,579,250
6227,PL,RSX6240,642000,6452250,X6227009,Platform-Brand Partn,R6154,INC,102,486,356,483"""
df = gdf(d)
df

,Entity,PL/BS,RSX Account,C Pack Account,G/L Account,PC,Profit Center Descr.,Trading partner,Trading partner Descr.,1,2,3,4
0,6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,#,Not assigned,366,328,5,481
1,6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R3020,DACH,984,659,475,894
2,6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R6001,SA,577,472,687,887
3,6154,BS,RSX4000,400100,400100D,X6154999,Bal. Sheet-6154,R6079,BX,416,400,159,432
4,6156,BS,RSX4000,400100,400100D,X6156999,Bal. Sheet-6156,R6154,INC,270,741,317,564
5,6157,BS,RSX4000,400100,400100D,X6157999,Bal. Sheet-6157,#,Not assigned,44,437,200,247
6,6157,BS,RSX4000,400100,400100D,X6157999,Bal. Sheet-6157,R6267,France,597,612,87,48
7,6157,BS,RSX4000,400100,400100S,X6157999,Bal. Sheet-6157,R6154,INC,844,251,800,624
8,6154,PL,RSX6240,640100,6408900,X6154999,Others,R6156,AUS,592,638,945,43
9,6156,PL,RSX6240,656000,6560000,X6156999,Other,R6157,UK,613,26,462,224


In [83]:
pd.read_csv('te.txt', engine='python', sep=r"\s*:\s*|\s+", header = None)

,0,1,2,3,4,5,6,7,8,9
0,Layer_00,3.46ms,Layer_01,3.40ms,Layer_02,3.56ms,Layer_03,3.49ms,Layer_04,3.44ms
1,Layer_05,3.45ms,Layer_06,3.44ms,Layer_07,3.46ms,Layer_08,3.45ms,Layer_09,3.48ms
2,Layer_10,3.48ms,Layer_11,3.48ms,Layer_12,3.57ms,Layer_13,3.54ms,Layer_14,3.60ms
3,Layer_15,3.58ms,Layer_16,3.59ms,Layer_17,3.57ms,Layer_18,3.55ms,Layer_19,3.57ms
4,Layer_20,3.58ms,Layer_21,3.57ms,Layer_22,3.52ms,Layer_23,3.56ms,Layer_24,3.47ms


In [92]:
nd = np.random.randn(20).reshape(5,4)
df = pd.DataFrame(nd)
df

,0,1,2,3
0,0.369586,-0.296591,1.544528,-0.053073
1,-0.452793,-0.634489,0.326829,2.656572
2,2.289719,0.023302,1.192873,-0.493804
3,-1.572314,0.447135,0.589677,0.920896
4,-0.374528,0.689214,2.619276,-0.417504


In [101]:
pd.Series(list(nd))

0    [0.3695861003591625, -0.29659099890013, 1.5445...
1    [-0.45279337510087897, -0.6344885684578799, 0....
2    [2.2897187079331984, 0.023302006872790094, 1.1...
3    [-1.5723135735310614, 0.4471350334417705, 0.58...
4    [-0.3745275102384259, 0.689213800121549, 2.619...
dtype: object

In [148]:
import pandas as pd
df = pd.DataFrame({'sales': [25, 20, 14]})
df

,sales
0,25
1,20
2,14


In [155]:
pd.read_csv('te.csv', engine='python', sep='\s+|,|\t+')

,COLID,TEMP1,TEMP2,TEMP3
0,1,20.1,40.2,8.5


In [109]:
df.sales.max()

25

In [157]:
df = pd.DataFrame({'A':[1000, 1000, 1000, 1000, 1000, 200, 200, 500, 500],
                   'B':[380, 380, 270, 270, 270, 45, 45, 45, 55],
                   'C':[380, 380, 270, 270, 270, 88, 88, 88, 88],
                   'D':[45, 32, 67, 89, 51, 90, 90, 90, 90]})
df

,A,B,C,D
0,1000,380,380,45
1,1000,380,380,32
2,1000,270,270,67
3,1000,270,270,89
4,1000,270,270,51
5,200,45,88,90
6,200,45,88,90
7,500,45,88,90
8,500,55,88,90


In [136]:
ga = df.groupby('A').D.transform('count') 
gb = df.groupby('B').D.transform('count')
gc = df.groupby('C').D.transform('count')

conditions = [
    ((ga > gb) & (ga > gc)),
    ((gb > ga) & (gb > gc)),
    ((gc > ga) & (gc > gb)),
]
choices = [df.groupby('A').D.transform('sum'), 
           df.groupby('B').D.transform('sum'),
           df.groupby('C').D.transform('sum')]
df['D'] = np.select(conditions, choices)
df

,A,B,C,D
0,1000,380,380,284
1,1000,380,380,284
2,1000,270,270,284
3,1000,270,270,284
4,1000,270,270,284
5,200,45,88,360
6,200,45,88,360
7,500,45,88,360
8,500,55,88,360


In [147]:
def get_count_sum(col, func):
    return df.groupby(col).D.transform(func)
    
ga = get_count_sum('A', 'count')
gb = get_count_sum('B', 'count')
gc = get_count_sum('C', 'count')


conditions = [
    ((ga > gb) & (ga > gc)),
    ((gb > ga) & (gb > gc)),
    ((gc > ga) & (gc > gb)),
]

choices = [get_count_sum('A', 'sum'), 
           get_count_sum('B', 'sum'),
           get_count_sum('C', 'sum')]

df['D'] = np.select(conditions, choices)
df

,A,B,C,D
0,1000,380,380,284
1,1000,380,380,284
2,1000,270,270,284
3,1000,270,270,284
4,1000,270,270,284
5,200,45,88,360
6,200,45,88,360
7,500,45,88,360
8,500,55,88,360


In [160]:
import re
with open('te.txt', 'r') as file:
    pairs = re.findall('(Layer_\d+)\s,\s(\d+\.\d+)ms', file.read())
pairs = [(k, float(v)) for k,v in pairs]
df = pd.DataFrame(pairs)
df.head()

,0,1
0,Layer_00,3.46
1,Layer_01,3.40
2,Layer_02,3.56
3,Layer_03,3.49
4,Layer_04,3.44


In [163]:
df = pd.read_csv('te.txt', engine='python', sep='\s+|\s+,\s+')
df

,,,,,,,,,,"C1,D1,E1,F1,G1,,,,,"
Layer_00,",","3.46ms,Layer_01",",","3.40ms,Layer_02",",","3.56ms,Layer_03",",","3.49ms,Layer_04",",",3.44ms
Layer_05,",","3.45ms,Layer_06",",","3.44ms,Layer_07",",","3.46ms,Layer_08",",","3.45ms,Layer_09",",",3.48ms
"C2,D2,E2,F2,G2,,,,,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None
Layer_00,",","3.42ms,Layer_01",",","3.39ms,Layer_02",",","3.51ms,Layer_03",",","3.41ms,Layer_04",",",3.43ms
Layer_05,",","3.40ms,Layer_06",",","3.43ms,Layer_07",",","3.45ms,Layer_08",",","3.43ms,Layer_09",",",3.42ms


In [181]:
with open('te.txt', 'r') as file:
    pairs = re.findall('(Layer_\d+)\s,\s(\d+\.\d+)ms', file.read())
pairs = [(k, float(v)) for k,v in pairs]
df = pd.DataFrame(pairs)
df

,0,1
0,Layer_00,3.46
1,Layer_01,3.40
2,Layer_02,3.56
3,Layer_03,3.49
4,Layer_04,3.44
5,Layer_05,3.45
6,Layer_06,3.44
7,Layer_07,3.46
8,Layer_08,3.45
9,Layer_09,3.48


In [182]:
df.groupby(0, as_index=False)['1'].apply(list)

KeyError: 'Column not found: 1'

In [183]:
d = """id,text,target
1,we always try to bring the heavy metal rt http.somethingsomething,1
4,on plus side look at the sky last night it was ablaze http.somethingsomething,1
6,inec office in abia set ablaze http.somethingsomething,1"""
df = gdf(d)
df

,id,text,target
0,1,we always try to bring the heavy metal rt http...,1
1,4,on plus side look at the sky last night it was...,1
2,6,inec office in abia set ablaze http.somethings...,1


In [184]:
df['text'] = df.text.replace((r'[http]*'),'')
df

,id,text,target
0,1,we always try to bring the heavy metal rt http...,1
1,4,on plus side look at the sky last night it was...,1
2,6,inec office in abia set ablaze http.somethings...,1


In [190]:
d="""id,text,target
1, metal rt http:\\something11,1
4,it was ablaze http:\\somethingdifferent,1
6,e http:\\itsjustaurl,1"""
df1 = gdf(d)
df1

,id,text,target
0,1,metal rt http:\something11,1
1,4,it was ablaze http:\somethingdifferent,1
2,6,e http:\itsjustaurl,1


In [202]:
df1.text.str.replace(r'https?:\S+', '')

<ipython-input-202-f395a5091b50>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df1.text.str.replace(r'https?:\S+', '')


0         metal rt 
1    it was ablaze 
2                e 
Name: text, dtype: object

In [214]:
d = """time,value
1:00,0
2:00,
3:00,0
4:00,1
5:00,
6:00,
7:00,1
8:00,1
9:00,0"""
df = gdf(d)
df

,time,value
0,1:00,0.0
1,2:00,NaN
2,3:00,0.0
3,4:00,1.0
4,5:00,NaN
5,6:00,NaN
6,7:00,1.0
7,8:00,1.0
8,9:00,0.0


In [217]:
df['value'] = df.value.where(df.value.diff().ne(0), '')
df

,time,value
0,1:00,0.0
1,2:00,NaN
2,3:00,0.0
3,4:00,1.0
4,5:00,NaN
5,6:00,NaN
6,7:00,1.0
7,8:00,
8,9:00,0.0


In [233]:
d = """Employee Name,Subjects,Billable,Hours,Date
Anu,Java,Yes,8,01-03-2021
Anu,Python,Yes,9,02-03-2021
Anu,SQL,No,6,03-03-2021
Anu,React,Yes,5,03-03-2021
Anu,.Net,No,8,04-03-2021
Bala,SQL,No,5,01-03-2021
Bala,Python,Yes,4,01-03-2021
Bala,Java,Yes,2,02-03-2021
Bala,.Net,No,8,03-03-2021
Bala,React,Yes,7,04-03-2021"""
df = gdf(d)
df

,Employee Name,Subjects,Billable,Hours,Date
0,Anu,Java,Yes,8,01-03-2021
1,Anu,Python,Yes,9,02-03-2021
2,Anu,SQL,No,6,03-03-2021
3,Anu,React,Yes,5,03-03-2021
4,Anu,.Net,No,8,04-03-2021
5,Bala,SQL,No,5,01-03-2021
6,Bala,Python,Yes,4,01-03-2021
7,Bala,Java,Yes,2,02-03-2021
8,Bala,.Net,No,8,03-03-2021
9,Bala,React,Yes,7,04-03-2021


In [234]:
df['Date'] = pd.to_datetime(df.Date, dayfirst=True)
df = df.set_index(df.Date).sort_index()
df['Utilization'] = (df['Hours']/9*100).round(2)
df['Billable Utilization'] = np.where(df.Billable.eq('Yes'), df['Hours']/9*100, np.nan)
df['Non Billable Utilization'] = np.where(df.Billable.eq('No'), df['Hours']/9*100, np.nan)
df = df[["Date", "Employee Name", "Utilization", "Billable Utilization", "Non Billable Utilization"]]
df

,Date,Employee Name,Utilization,Billable Utilization,Non Billable Utilization
Date,,,,,
2021-03-01,2021-03-01,Anu,88.89,88.888889,NaN
2021-03-01,2021-03-01,Bala,55.56,NaN,55.555556
2021-03-01,2021-03-01,Bala,44.44,44.444444,NaN
2021-03-02,2021-03-02,Anu,100.00,100.000000,NaN
2021-03-02,2021-03-02,Bala,22.22,22.222222,NaN
2021-03-03,2021-03-03,Anu,66.67,NaN,66.666667
2021-03-03,2021-03-03,Anu,55.56,55.555556,NaN
2021-03-03,2021-03-03,Bala,88.89,NaN,88.888889
2021-03-04,2021-03-04,Anu,88.89,NaN,88.888889


In [238]:
d = """ID,DAY,LABEL
AA,01/26/2020,TRUE
AA,01/27/2020,FALSE
BB,01/28/2020,TRUE
BB,01/29/2020,FALSE
BB,01/30/2020,FALSE"""
df1 = gdf(d)
df1['DAY'] = pd.to_datetime(df1.DAY)
df1


,ID,DAY,LABEL
0,AA,2020-01-26,True
1,AA,2020-01-27,False
2,BB,2020-01-28,True
3,BB,2020-01-29,False
4,BB,2020-01-30,False


In [244]:
d = """ID,DAY,VAL
AA,01/26/2020,50
AA,01/27/2020,60
BB,01/28/2020,200"""
df2 = gdf(d)
df2['DAY'] = pd.to_datetime(df2.DAY)
df2

,ID,DAY,VAL
0,AA,2020-01-26,50
1,AA,2020-01-27,60
2,BB,2020-01-28,200


In [250]:
df = pd.merge_asof(df1, df2, on='DAY', by='ID').assign(VAL=df['VAL'].mask(~df['LABEL'])
#df['VAL'] = np.where(df.LABEL,df.VAL,np.nan)
#df['VAL']=df.VAL.mask(~df['LABEL'])
df

SyntaxError: invalid syntax (<ipython-input-250-38f4ca3be1b3>, line 4)

In [242]:
df1.merge(df2, on=[ID, DAY], how='left')

NameError: name 'ID' is not defined

In [251]:
df1.merge(df2.assign(LABEL=True), how='left', on=['ID', 'DAY', 'LABEL'])

,ID,DAY,LABEL,VAL
0,AA,2020-01-26,True,50.0
1,AA,2020-01-27,False,NaN
2,BB,2020-01-28,True,200.0
3,BB,2020-01-29,False,NaN
4,BB,2020-01-30,False,NaN


In [255]:
pd.to_datetime("3000.01.01", errors='coerce')

NaT

In [256]:
pd.Timestamp.max

Timestamp('2262-04-11 23:47:16.854775807')

In [257]:
import qpython

ModuleNotFoundError: No module named 'qpython'